In [5]:
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas_datareader import data
from dask.distributed import Client
from dask.distributed import as_completed
import openpyxl
import xlsxwriter
import pandas_datareader.data as web
import statsmodels.tsa.stattools as ts
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import pandas_market_calendars as mcal
import os
import sqlite3
import matplotlib.pyplot as plt
import dataframe_image as dfi
import seaborn as sns
import scipy.stats as stats
import pickle

import datetime as dt

import scipy

import sys

import os

path = os.getcwd()

print(path)

C:\Users\User\Documents\wiki\wiki\dev\python\Python-Stock\code\Screener\code


In [6]:

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [7]:
def trend_tau(set_):
    [x1, x2] = set_
    tau, p_value = stats.kendalltau(x1,x2)
    return([tau,p_value])


In [8]:
def split_sequences(sequences, n_steps_in):

    X = list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        # check if we are beyond the dataset
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix, :]
        X.append(seq_x)

    return np.array(X)

In [9]:
def highlight_cells(cell):
    #def inner(cell):
    color = 'green' if cell == True else ''
    return 'background-color: {}'.format(color)
    #return inner

In [10]:
end_ = datetime.today()
#start = end - relativedelta(years=2) - relativedelta(days=3)
start_ = end_ - relativedelta(months=64)
start_X2_ = end_ - relativedelta(months=64*2)

nyse = mcal.get_calendar('NYSE')
nyse_dates = nyse.schedule(start_date=start_, end_date=end_)

nyse_X2 = mcal.get_calendar('NYSE')
nyse_dates_X2 = nyse.schedule(start_date=start_X2_, end_date=end_)

start = nyse_dates.index[0]
start_X2 = nyse_dates_X2.index[0]

end = nyse_dates.index[-1]
print(start.strftime('%Y-%m-%d'))
print(end.strftime('%Y-%m-%d'))

#used for trend
n_trend = 50

2017-04-17
2022-08-15


In [11]:
"""
db_filename = '.\\data\\interim\\todo.db'

db_is_new = not os.path.exists(db_filename)

conn = sqlite3.connect(db_filename)

if db_is_new:
    print('Need to create schema')
else:
    print('Database exists, assume schema does, too.')

conn.close()
"""

"\ndb_filename = '.\\data\\interim\\todo.db'\n\ndb_is_new = not os.path.exists(db_filename)\n\nconn = sqlite3.connect(db_filename)\n\nif db_is_new:\n    print('Need to create schema')\nelse:\n    print('Database exists, assume schema does, too.')\n\nconn.close()\n"

In [12]:
dict_sectors = {'Basic Materials':'VAW', 'Communication Services':'VOX', 'Consumer Cyclical':'VCR',
       'Consumer Defensive':'VDC', 'Energy':'VDE', 'Financial Services':'VFH', 'Healthcare':'VHT',
       'Industrials':'VIS', 'Real Estate':'VNQ', 'Technology':'VGT', 'Utilities':'VPU', 'error':'error'}

dict_indexes = {'SPTM':'SP1500', 'SPY':'SP500', 'SLY':'SP600', 'MDYG':'SP400','error':'error'}

invert_dict_indexes = {v: k for k, v in dict_indexes.items()}

dict_sectors_reverse = {'VAW':'Basic Materials',
 'VOX':'Communication Services',
 'VCR':'Consumer Cyclical',
 'VDC':'Consumer Defensive',
 'VDE':'Energy',
 'VFH':'Financial Services',
 'VHT':'Healthcare',
 'VIS':'Industrials',
 'VNQ':'Real Estate',
 'VGT':'Technology',
 'VPU':'Utilities',
 'error': 'error'}

In [13]:
index_names = ['SP1500', 'SP500', 'SP600', 'SP400']
indexes = ['SPTM','SPY','SLY','MDYG']
sector_names = ['Communication Services ETF', 'Consumer Discretionary ETF', 'Consumer Staples ETF', 'Energy ETF', 'Financials ETF', 'Health Care ETF', 'Industrials ETF', 'Information Technology ETF', 'Materials ETF', 'Real Estate ETF', 'Utilities ETF']
sectors = ['VOX','VCR','VDC','VDE','VFH','VHT','VIS','VGT','VAW','VNQ','VPU']

etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

#M1_MONEY_MULTIPLIER = M1NS/BOGMBASE

FRED_Indicators = [
"ASPUS",\
"AWHAETP",\
"B432RG3A086NBEA",\
"BAA10Y",\
"BACDINA066MNFRBNY",\
"BACTSAMFRBDAL",\
"BAMLC0A0CM",\
"BAMLCC0A1AAATRIV",\
"BAMLCC0A4BBBTRIV",\
"BAMLH0A3HYC",\
"BAMLHYH0A0HYM2TRIV",\
"BOGMBASE",\
"BOGZ1FA105015103Q",\
"BOGZ1FA145050005Q",\
"BOGZ1FA205050005Q",\
"BOGZ1FA315050005Q",\
"BOGZ1FA385050005Q",\
"BOGZ1FL105015105Q",\
"BUSAPPWNSACA",\
"BUSLOANS",\
"CASTHPI",\
"CES0500000003",\
"CES4348400001",\
"CFNAI",\
"CFNAIDIFF",\
"CFSBCACTIVITYMFG",\
"CILACBQ158SBOG",\
"CIVPART",\
"COMREPUSQ159N",\
"CONSUMER",\
"CP",\
"CPALTT01USQ657N",\
"CPIAUCSL",\
"CSCICP03USM665S",\
"CSUSHPINSA",\
"CUSR0000SEHA",\
"CUUR0000SEHA",\
"DALLCIACBEP",\
"DCOILBRENTEU",\
"DCOILWTICO",\
"DEXCHUS",\
"DEXUSUK",\
"DFF",\
"DFII10",\
"DGORDER",\
"DGS1",\
"DGS10",\
"DGS1MO",\
"DGS2",\
"DGS30",\
"DGS3MO",\
"DGS5",\
"DRBLACBS",\
"DRCCLACBS",\
"DRSFRMACBS",\
"DGS3MO",\
"DTWEXBGS",\
"EMRATIO",\
"ERENTUSQ176N",\
"ETOTALUSQ176N",\
"FEDFUNDS",\
"FGCCSAQ027S",\
"FPCPITOTLZGUSA",\
"GASREGW",\
"GDPC1",\
"GFDEBTN",\
"GFDEGDQ188S",\
"GOLDAMGBD228NLBM",\
"GVZCLS",\
"HDTGPDUSQ163N",\
"HOSMEDUSM052N",\
"IC4WSA",\
"ICSA",\
"IIPUSNETIQ",\
"INDPRO",\
"INTDSRUSM193N",\
"IRLTLT01USM156N",\
"KCFSI",\
"LES1252881600Q",\
"LEU0252918500Q",\
"LFWA64TTUSM647S",\
"LRUN64TTUSQ156S",\
"LXXRCSA",\
"M1",\
"M1V",\
"M1NS",\
"M2V",\
"MABMM301USM189S",\
"MANMM101USA189S",\
"MEFAINUSA672N",\
"MEHOINUSA672N",\
"MEPAINUSA672N",\
"MICH",\
"MOGMBASE",\
"MORTGAGE30US",\
"MPRIME",\
"MSPUS",\
"MVMTD027MNFRBDAL",\
"MZMSL",\
"NEWORDER",\
"NFCI",\
"NROU",\
"NYSTHPI",\
"PAYEMS",\
"PCE",\
"PERMIT",\
"POP",\
"POPTOTUSA647NWDB",\
"PPIACO",\
"PRFI",\
"PSAVERT",\
"Q10096USQ144NNBR",\
"RECPROUSM156N",\
"RSAHORUSQ156S",\
"SPCS20RSA",\
"STLFSI2",\
"T10Y2Y",\
"T10Y3M",\
"T10YIE",\
"T5YIE",\
"T5YIFR",\
"TB3MS",\
"TCU",\
"TDSP",\
"TEDRATE",\
"TOTALSA",\
"TREAST",\
"TTLHH",\
"TWEXB",\
"UMCSENT",\
"U6RATE",\
"UNRATE",\
"USALOLITONOSTSAM",\
"USPHCI",\
"USREC",\
"USROA",\
"USROE",\
"USSLIND",\
"USSTHPI",\
"VIXCLS",\
"VXVCLS",\
"WDFUELLA",\
"WGS3MO",\
"WILLLRGCAPVAL",\
"WILLMICROCAPPR",\
"WPU0911",\
"WPUSI019011",\
"WPU101",\
"WTB3MS"    
]

#volume
dict_treasury = {
'TreasuryYield13':'^IRX',\
'T bill Euro F':  'GE=F',\
'Two Year T F': 'ZT=F',\
'Five-Year US Treasury Note Futu':'ZF=F',\
'Ten Year T F': 'ZN=F',\
'U.S. Treasury Bond Futures':'ZB=F',\
}

#TVV
currencies = ['EURUSD=X','GBPUSD=X','AUDUSD=X','BRLUSD=X','JPY=X','GE=F','ZAR=X','CNYUSD=X']

dict_currencies_futures = {
    'World:USD':'UUP',\
    'Euro:USD':'EURUSD=X',\
    'Pound:USD':'GBPUSD=X',\
    'Australia:USD':'AUDUSD=X',
    'Real:USD':'BRLUSD=X',\
    'Yen:USD':'JPY=X',\
    'EuroDollar':'USD',\
    'Rand:USD':'ZAR=X',\
    'Yuan:USD':'CNYUSD=X',
    'Peso:USD':'MXNUSD=X'
}

#doesn't work ,'B0=F'

indexes_futures = ['^RUT','^IRX','^GSPC']

dict_indexes_w_futures = {
    'Russel2000':'^RUT',\
    'Mini Dow Jones Indus.':'YM=F',\
    'Nasdaq 100':'NQ=F',\
    'SP500': 'ES=F',\
    'SP500':'^GSPC'
}

#'SIL=F','MGC=F','RTY=F'

other_commodities_list = ['YM=F','NQ=F','ZB=F','ZF=F','BZ=F','ZO=F','KE=F','ZR=F','ZM=F','ZL=F','GF=F','CT=F','LBS=F','OJ=F']

dict_materials_futures = {
    'Cotton':'CT=F',\
    'Random Length Lumber Futures':'LBS=F',\
}

#price
#currencies = ['EURUSD=X','GBPUSD=X','AUDUSD=X','BRLUSD=X','JPY=X','GE=F']

dict_foods = {
'Corn': 'ZC=F',\
'Soybeans': 'ZS=F',\
'Chicago Wheat': 'ZW=F',\
'Live Cattle': 'LE=F',\
'Lean Hogs': 'HE=F',\
'Cocoa': 'CC=F',\
'Sugar': 'SB=F',\
'Coffee': 'KC=F',\
'Oat Futures':'ZO=F',\
'KC HRW Wheat Futures':'KE=F',\
'Rough Rice Futures':'ZR=F',\
'Soybean Meal Futures':'ZM=F',\
'Soybean Oil Futures':'ZL=F',\
'Feeder Cattle Futures':'GF=F',\
'Orange Juice':'OJ=F',
}

#price
dict_metals = {'Gold': 'GC=F',\
'Platinum':  'PL=F',\
'Palladium': 'PA=F',\
'Silver': 'SI=F',\
'Copper':  'HG=F',\
'Aluminium': 'ALI=F'
}

#price
dict_energy = {
'Crude Oil':  'CL=F',\
'Brent Crude Oil Last Day Financ':'BZ=F',\
'Heating Oil': 'HO=F',\
'Natural Gas': 'NG=F',\
'RB Gasoline': 'RB=F'
}

etf_bonds = [*etf_bonds,*etf_muni_bonds,*etf_treasuries]

In [14]:
SP500ticker_list = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
SP600ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1]
SP400ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0]


In [15]:
sp500 = SP500ticker_list['Symbol']
sp500 = [w.replace('.', '-') for w in sp500]
sp600 = SP600ticker_list['Ticker symbol']
sp600 = [w.replace('.', '-') for w in sp600]
sp400 = SP400ticker_list['Ticker symbol']
sp400 = [w.replace('.', '-') for w in sp400]


In [16]:
len(sp500)+len(sp600)+len(sp400)

1505

In [17]:
print(len(sp500))
SP1500 = list()
SP1500.extend(sp500)
sp600 = list(set(sp600).difference(SP1500))
print(len(sp600))
SP1500.extend(sp600)
sp400 = list(set(sp400).difference(SP1500))
print(len(sp400))
SP1500.extend(set(sp400).difference(SP1500))

503
601
401


In [18]:
#>.5The time series is mean reverting
#=.5 time series is a Geometric Brownian Motion
#<.5 time series is trending

def hurst(ts):
    """
    Returns the Hurst Exponent of the time series vector ts

    Parameters
    ----------
    ts : `numpy.array`
        Time series upon which the Hurst Exponent will be calculated

    Returns
    -------
    'float'
        The Hurst Exponent from the poly fit output
    """
    # Create the range of lag values
    lags = range(2, 100)

    # Calculate the array of the variances of the lagged differences
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]

    # Use a linear fit to estimate the Hurst Exponent
    poly = polyfit(log(lags), log(tau), 1)

    # Return the Hurst exponent from the polyfit output
    return poly[0]*2.0

In [19]:
sp1500_index_df = pd.DataFrame()

lists = ["sp500","sp600","sp400"]

for l in lists:
    if(l=='sp500'):
        df = pd.DataFrame(sp500,columns=['Symbol'])
        print(len(df))
        #range_ = 
        #print(range_)
        df['Market Index'] = pd.Series(["SP500" for x in range(0,len(df.index),1)])
    elif(l=='sp600'):
        df = pd.DataFrame(np.array(sp600),columns=['Symbol'])        
        print(len(df))        
        df['Market Index'] = pd.Series(["SP600" for x in range(len(df.index))])
        df.index = range(len(sp500),(len(sp500)+len(df.index)),1)        
    else:
        df = pd.DataFrame(np.array(sp400),columns=['Symbol'])
        print(len(df))
        df['Market Index'] = pd.Series(["SP400" for x in range(len(df.index))])
        df.index = range((len(sp500)+len(sp600)),(len(sp500)+len(sp600)+len(df.index)),1)
    sp1500_index_df = pd.concat([sp1500_index_df,df],axis=0)

503
601
401


In [20]:
len(sp1500_index_df)-len(SP1500)

0

In [21]:
def Fred_Data(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    nyse_dates_ = npa[3]
    
    temp = web.DataReader(str(name), 'fred', start_, end_)
    #temp = web.DataReader(npa[1][0],'fred',start,end)
    temp = temp.asfreq('D').interpolate(method='time',limit_direction='forward',limit_area='inside').reindex(nyse_dates_.index)
    temp.columns = ['Value']
    #temp.index = pd.to_datetime(temp.index)
    
    #temp['Symbol'] = name
    
    #print(temp)
    #temp_new = temp.asfreq('D').interpolate(method='time')    
    temp_dates = pd.date_range(temp.index[0].strftime('%Y-%m-%d'), temp.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp.reindex(nyse_inbetween_temp_dates)
    temp_new['Symbol'] = name
    
    #temp = temp.resample(frequency).mean().dropna()
    return(temp_new)
    #return(temp)

In [22]:

def dl(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    nyse_dates_ = npa[3]
    subset = yf.download(name, start=start_, end=end_, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    
    temp_new = subset.asfreq('D').interpolate(method='time',limit_area='inside').reindex(nyse_dates_.index)
    temp_dates = pd.date_range(subset.index[0].strftime('%Y-%m-%d'), subset.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp_new.reindex(nyse_inbetween_temp_dates)
    subset_ = temp_new
    subset_['Symbol'] = name

    return (subset_)
    
def dl2(assets):

    yahoo_financials = YahooFinancials(assets)

    data = yahoo_financials.get_historical_price_data(start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'), time_interval='daily')
    return(data)

In [23]:
def getStock(npa):
    symbol = npa[0]
    start_=npa[1]
    end_=npa[2]
    
    data_ = yf.download(symbol, start=start_,end=end_)
  
    return([symbol,data_])

In [24]:
def getTickerData(npa):
    symbol = npa[0]
    
    stock_data = yf.Ticker(symbol) 
    return([symbol,stock_data])

In [25]:
def getSector(npa):
    #position = npa[0]
    symbol = npa[0]
    #start_=npa[2]
    #end_=npa[3]
    
    tickerdata = yf.Ticker(symbol)
    
    if 'sector' in tickerdata.info:
        sector = tickerdata.info['sector']
    else:
        sector = 'error'

    if 'industry' in tickerdata.info:
        industry = tickerdata.info['industry']
    else:
        industry = 'error'

    return([symbol,sector,industry])

In [26]:
def derive_price_supply_trends(npa_):
    name = npa_[0]
    data = npa_[1]
    nyse_dates_ = npa_[2]
        
    temp_new = data.asfreq('D').interpolate(method='time',limit_area='inside').reindex(nyse_dates_.index)
    temp_dates = pd.date_range(data.index[0].strftime('%Y-%m-%d'), data.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp_new.reindex(nyse_inbetween_temp_dates)
    
    data_ = temp_new
    data_['risk_50td_trend'] = (data.shift(50)-data)/ ((data.rolling(50).max()-data.rolling(50).min()) /4) 
    data_['Symbol'] = name
    return(data_)
    

In [27]:
def derive_tvv(npa_):
    name = npa_[0]
    data = npa_[1]
    #print(data)
    nyse_dates_ = npa_[2]
        
    temp_new = data.asfreq('D').interpolate(method='time',limit_area='inside').reindex(nyse_dates_.index)
    temp_dates = pd.date_range(data.index[0].strftime('%Y-%m-%d'), data.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
    nyse_inbetween_temp_dates = nyse_dates_.index[(nyse_dates_.index >= temp_dates[0]) & (nyse_dates_.index <= temp_dates[-1])]
    temp_new = temp_new.reindex(nyse_inbetween_temp_dates)
    
    data_ = temp_new
    data_['Symbol'] = name

    """
    if(np.sum(duplicates==names_prices[n])>0):
        #potential duplicate
        if((np.sum((np.unique(prices_df['Symbol']))==n))>0):
            pass
        else:
            prices_df = pd.concat([prices_df,data_],axis=0)            
    else:
    """
    
    data_['VWAP'] = (data_['Adj Close']*data_['Volume'])
    
    data_['buy_volume_pct'] = (data_['Adj Close']-data_['Low'])/(data_['High']-data_['Low'])
    data_['sell_volume_pct'] = (data_['Adj Close']-data_['High'])/(data_['High']-data_['Low'])



    data_['daily_buy_vol'] = data_['buy_volume_pct']*data_['Volume']
        
        
    
    data_['daily_sell_vol'] = data_['sell_volume_pct']*data_['Volume']
    
    data_['net_daily_vol'] = data_['daily_buy_vol']+data_['daily_sell_vol']
    
    data_['20td_trend_vol'] = data_['net_daily_vol'].rolling(20).sum()
    
    
    data_['20td_tot_daily_vol'] = data_['Volume'].rolling(20).sum()
    
    data_['20td_trend_vol_factor'] = data_['20td_trend_vol']/data_['20td_tot_daily_vol']
    
    data_['20td_tvf_rng_div_2'] = (data_['20td_trend_vol_factor'].rolling(20).max()-data_['20td_trend_vol_factor'].rolling(20).min())/2
    
    data_['20td_tvf_vel'] = data_['20td_trend_vol_factor']/data_['20td_tvf_rng_div_2']    
    
    
    data_['50td_trend_vol'] = data_['net_daily_vol'].rolling(20).sum()
    
    data_['50td_tot_daily_vol'] = data_['Volume'].rolling(20).sum()
    
    data_['50td_trend_vol_factor'] = data_['50td_trend_vol']/data_['50td_tot_daily_vol']
    
    data_['50td_tvf_rng_div_4'] = (data_['50td_trend_vol_factor'].rolling(20).max()-data_['50td_trend_vol_factor'].rolling(20).min())/4
    
    data_['50td_tvf_vel'] = data_['50td_trend_vol_factor']/data_['50td_tvf_rng_div_4']    
    
    
    data_['trailing_1yr_max'] = (pd.DataFrame(data_)['High']).rolling(252).max()
    data_['MIN_Lookback_One_Year'] = data_['trailing_1yr_max'].rolling(252).min()
    #data_['MIN_Lookback_6_Months'] = data_['trailing_1yr_max'].rolling(126).min()
    data_['MIN_Lookback_2_Months'] = data_['trailing_1yr_max'].rolling(42).min()
    data_['MIN_Lookback_1_Months'] = data_['trailing_1yr_max'].rolling(21).min()

    data_['trailing_1yr_min'] = (pd.DataFrame(data_)['Low']).rolling(252).min()
    
    data_['supply_trend_1yr'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(252).min()
    
    #data_['supply_trend_180d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(126).min()
    
    #data_['supply_trend_200td'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(63).min()
    
    data_['supply_trend_60d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(42).min()
    
    data_['supply_trend_30d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(21).min()

    data_['trailing_60d_max'] = (pd.DataFrame(data_)['High']).rolling(42).max()
    data_['trailing_60d_min'] = (pd.DataFrame(data_)['Low']).rolling(42).min()
    
    data_['trailing_30d_max'] = (pd.DataFrame(data_)['High']).rolling(21).max()
    data_['trailing_30d_min'] = (pd.DataFrame(data_)['Low']).rolling(21).min()
    
    data_['200tdSMA'] = (pd.DataFrame(data_)['Adj Close']).rolling(200).mean()
    
    data_['200tdSDev'] = (pd.DataFrame(data_)['Adj Close']).rolling(200).std()
    
    data_['TP'] = (data_['Adj Close'] + data_['Low'] + data_['High'])/3
    data_['50tdstdTP'] = data_['TP'].rolling(50).std(ddof=0)
    data_['50tdMA-TP'] = data_['TP'].rolling(50).mean()
    data_['50tdBOLU'] = data_['50tdMA-TP'] + 2*data_['50tdstdTP']
    data_['50tdBOLD'] = data_['50tdMA-TP'] - 2*data_['50tdstdTP']    

    data_['trailing_risk_40d_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()
    data_['trailing_risk_40d_min'] = (pd.DataFrame(data_)['Low']).rolling(40).min()
    
    data_['risk_per_share'] = data_['trailing_risk_40d_max']-data_['trailing_risk_40d_min']
    

    data_['30d_vol'] = (pd.DataFrame(data_)['Volume']).rolling(21).sum()
        
    data_['30d_vol_2yr'] = data_['30d_vol'].rolling(512).mean()
    data_['volume_factor'] = data_['30d_vol']/data_['30d_vol_2yr']
    
    #data_['1YMF'] = (data_['Adj Close']-data_['MIN_Lookback_One_Year']) + (data_['Adj Close']-data_['MIN_Lookback_6_Months']) + (data_['Adj Close']-data_['MIN_Lookback_2_Months']) + (data_['Adj Close']-data_['MIN_Lookback_1_Months'])
    data_['1YMF'] = (data_['Adj Close']-data_['MIN_Lookback_One_Year']) + (data_['Adj Close']-data_['MIN_Lookback_2_Months']) + (data_['Adj Close']-data_['MIN_Lookback_1_Months'])
    
    data_['risk_trend_factor'] = data_['1YMF']/data_['risk_per_share']
    
    return(data_)

In [28]:
def calculateMetrics(data_):

    df_line = data_[['Symbol','buy_volume_pct','sell_volume_pct','daily_buy_vol','daily_sell_vol','net_daily_vol','20td_trend_vol','20td_tot_daily_vol','20td_trend_vol_factor','20td_tvf_rng_div_2','20td_tvf_vel','50td_trend_vol','50td_tot_daily_vol','50td_trend_vol_factor','50td_tvf_rng_div_4','50td_tvf_vel','MIN_Lookback_One_Year', 'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months','risk_trend_factor','risk_per_share','volume_factor','Adj Close','50tdBOLD','50tdMA-TP','50tdBOLU','200tdSMA','200tdSDev']].iloc[[-1]].reset_index().set_index('Symbol')
    name = df_line.index[0]
    adf = ts.adfuller(data_['Adj Close'].tail(n_trend).values)[1]

    hurst_ = hurst(data_['Adj Close'].tail(n_trend).values)    

    data = data_['Adj Close'].tail(n_trend)
    dates = [pd.to_datetime(d).value for d in data.index]

    dates = np.array(dates)-np.array(dates[0])/10**11

    int_date_data = pd.concat([pd.DataFrame(dates),pd.DataFrame(data.values)],axis=1)

    trend_t = [*[trend_tau([pd.DataFrame(int_date_data.iloc[:,0]).values,data.values])]]

    tau = trend_t[0][0]
    tau_p = trend_t[0][1]

    try:
        adf = ts.adfuller(data_['Adj Close'].tail(n_trend).values)[1]
    except:
        adf = 'error'

    try:
        hurst_ = hurst(data_['Adj Close'].tail(n_trend).values)
    except:
        hurst_ = 'error'

    temp = pd.concat([df_line,pd.DataFrame([adf],index=[name],columns=['adf_50td']),pd.DataFrame([tau],index=[name],columns=['tau_50td']),pd.DataFrame([tau_p],index=[name],columns=['tau_50td_p']),pd.DataFrame([hurst_],index=[name],columns=['hurst_50td'])],axis=1)
    temp.columns = ['Date',*temp.columns[1:]]
    temp

    return(temp)
    

In [29]:
def extractTickerValues(npa):
    
    n_ = npa[0]
    r_ = npa[1]
    #print(r_)
    print(n_)

    subset = pd.DataFrame([r_.info])

    others = ['quarterly_balance_sheet','quarterly_cashflow','quarterly_dividends','quarterly_earnings','quarterly_financials','mutualfund_holders','options','sustainability']

    o_items = []

    #print(r_)
    for o in others:
        if(np.sum(np.array(dir(r_))==[o])>0):
            o_item = getattr(r_, o)
            o_items.append([o,o_item])            

    
    values = pd.DataFrame()

    trackers = ['shortName','beta','currentRatio','debtToEquity','dividendRate','dividendYield','earningsGrowth','earningsQuarterlyGrowth','ebitda','ebitdaMargins','exDividendDate','forwardEPS','forwardPE','freeCashflow','grossMargins','grossProfits','heldPercentInstitutions','industry','lastDividendDate','lastDividendValue','lastSplitDate','lastSplitFactor','marketCap','operatingCashflow','operatingMargins','payoutRatio','pegRatio','priceToBook','priceToSalesTrailing12Months','profitMargins','quickRatio','returnOnAssets','returnOnEquity','revenueGrowth','revenuePerShare','sector','sharesOutstanding','shortRatio','totalAssets','totalCash','totalCashPerShare','totalDebt','totalRevenue','trailingEPS','trailingAnnualDividendRate','trailingAnnualDividendYield','trailingPE']

    for t in trackers:
        if(np.sum(np.where(subset.columns==t))==0):
            values = pd.concat([values,pd.DataFrame('error',index=[n_],columns=[t])],axis=1)
        else:
            values = pd.concat([values,pd.DataFrame(subset[t].values[0],index=[n_],columns=[t])],axis=1)

    return([n_,values,o_items])

In [30]:
tvv_extracted_list = np.unique([*[dict_indexes_w_futures[d] for d in dict_indexes_w_futures],*[dict_materials_futures[d] for d in dict_materials_futures],*[dict_foods[d] for d in dict_foods],*[dict_currencies_futures[d] for d in dict_currencies_futures],*[dict_treasury[d] for d in dict_treasury],*[dict_metals[d] for d in dict_metals],*[dict_energy[d] for d in dict_energy]])
npa = []
for f in tvv_extracted_list:
    npa.append([f,start_X2,end])

print("npa length:getStock(commodites):",len(npa))

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()
future = client.map(getStock, npa)
results_commodities_futures = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_commodities_futures.append("error")
    else:
        results_commodities_futures.append(f.result()) 
client.close()

"""
data_['trailing_risk_40d_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()
data_['trailing_risk_40d_min'] = (pd.DataFrame(data_)['Low']).rolling(40).min()
"""

npa length:getStock(commodites): 47


C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\distributed\client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.10.6 | None      | None    |
| lz4     | 4.0.2  | 3.1.10    | 3.1.10  |
| numpy   | 1.23.1 | 1.22.4    | 1.22.4  |
| tornado | 6.2    | 6.1       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


"\ndata_['trailing_risk_40d_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()\ndata_['trailing_risk_40d_min'] = (pd.DataFrame(data_)['Low']).rolling(40).min()\n"

In [31]:
#price_list = [*[dict_currencies_futures[d] for d in dict_currencies_futures],*[dict_treasury[d] for d in dict_treasury],*[dict_metals[d] for d in dict_metals],*[dict_energy[d] for d in dict_energy]]


In [32]:
names_comm_results = [r[0] for r in results_commodities_futures]
sum_comm_tvv_volume = [np.sum(r[1]['Volume']) for r in results_commodities_futures]

In [33]:
no_volume_comm = np.array(names_comm_results)[np.array(sum_comm_tvv_volume)==0]


In [34]:
#other_commodities_list_df[other_commodities_list_df['Symbol']=='ALI=F']

In [35]:
list_comm_output = [pd.DataFrame(r[1]) for r in results_commodities_futures]
list_comm_prices = [r[0] for r in results_commodities_futures]
dict_comm_prices = dict(zip(*[list_comm_prices,list_comm_output])) 


In [36]:
"""
full_commodities_price_df_pvt = pd.DataFrame()

for r_df in results_commodities_futures:

        temp_df = r_df[1].asfreq('D').interpolate(method='time',limit_direction='forward',limit_area='inside').reindex(nyse_dates_X2.index)
        temp_dates = pd.date_range(temp_df.index[0].strftime('%Y-%m-%d'), temp_df.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
        nyse_inbetween_temp_dates = nyse_dates_X2.index[(nyse_dates_X2.index >= temp_dates[0]) & (nyse_dates_X2.index <= temp_dates[-1])]
        temp_new = temp_df.reindex(nyse_inbetween_temp_dates)
        #temp_new['Symbol'] = r_df[0]
        price = temp_new[['Adj Close']]
        price.columns = [r_df[0]]
        print(price)
        full_commodities_price_df_pvt = pd.concat([price.copy(),full_commodities_price_df_pvt],axis=1)
        
#full_commodities_df_pvt = pd.pivot_table(full_commodities_df_pvt, values=['Adj Close'], index=other_commodities_list_df.index,columns=['Symbol'])
#full_commodities_df_pvt.columns = full_commodities_df_pvt.columns.droplevel(0)        
"""

"\nfull_commodities_price_df_pvt = pd.DataFrame()\n\nfor r_df in results_commodities_futures:\n\n        temp_df = r_df[1].asfreq('D').interpolate(method='time',limit_direction='forward',limit_area='inside').reindex(nyse_dates_X2.index)\n        temp_dates = pd.date_range(temp_df.index[0].strftime('%Y-%m-%d'), temp_df.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))\n        nyse_inbetween_temp_dates = nyse_dates_X2.index[(nyse_dates_X2.index >= temp_dates[0]) & (nyse_dates_X2.index <= temp_dates[-1])]\n        temp_new = temp_df.reindex(nyse_inbetween_temp_dates)\n        #temp_new['Symbol'] = r_df[0]\n        price = temp_new[['Adj Close']]\n        price.columns = [r_df[0]]\n        print(price)\n        full_commodities_price_df_pvt = pd.concat([price.copy(),full_commodities_price_df_pvt],axis=1)\n        \n#full_commodities_df_pvt = pd.pivot_table(full_commodities_df_pvt, values=['Adj Close'], index=other_commodities_list_df.index,columns=['Symbol'])

In [37]:

price_comm_df_pvt = pd.DataFrame()

for d in dict_comm_prices:
    temp = dict_comm_prices[d]
    temp['Symbol'] = d
    price_comm_df_pvt = pd.concat([price_comm_df_pvt,temp],axis=0)

price_comm_df_pvt = pd.pivot_table(price_comm_df_pvt, values=['Adj Close'], index=price_comm_df_pvt.index,columns=['Symbol']).interpolate(limit_area='inside', method='time')
#price_comm_df_pvt.columns = price_comm_df_pvt.columns.droplevel(0)
#price_comm_df_pvt.columns = [d for d in dict_comm_prices]

#9 quarters * 5 = 11.25 years * .9 ~ 10.25 years
price_comm_df_10_yr_pvt = price_comm_df_pvt.dropna(axis=1, thresh = int(0.90*price_comm_df_pvt.shape[0]))
price_comm_df_10_yr_pvt.columns = price_comm_df_10_yr_pvt.columns.droplevel(0)
price_comm_df_5_yr_pvt = price_comm_df_pvt.tail(int(len(price_comm_df_10_yr_pvt)/2))
price_comm_df_5_yr_pvt.columns = price_comm_df_5_yr_pvt.columns.droplevel(0)
price_comm_df_5_yr_pvt = price_comm_df_5_yr_pvt.dropna(axis=1, thresh = int(0.95*price_comm_df_5_yr_pvt.shape[0]))
price_comm_df_2_yr_pvt = price_comm_df_pvt.tail(int(len(price_comm_df_10_yr_pvt)/5))
price_comm_df_2_yr_pvt.columns = price_comm_df_2_yr_pvt.columns.droplevel(0)
price_comm_df_2_yr_pvt = price_comm_df_2_yr_pvt.dropna(axis=1, thresh = int(0.95*price_comm_df_2_yr_pvt.shape[0]))


price_comm_df_10_yr_pvt_corr = price_comm_df_10_yr_pvt.asfreq('D').interpolate(method='time').asfreq('W-'+price_comm_df_10_yr_pvt.index[-1].strftime('%a')).corr()

price_comm_df_5_yr_pvt_corr = price_comm_df_5_yr_pvt.asfreq('D').interpolate(method='time').asfreq('W-'+price_comm_df_5_yr_pvt.index[-1].strftime('%a')).corr()

price_comm_df_2_yr_pvt_corr = price_comm_df_2_yr_pvt.asfreq('D').interpolate(method='time').asfreq('W-'+price_comm_df_2_yr_pvt.index[-1].strftime('%a')).corr()





In [38]:
"""
for d in [d for d in dict_comm_prices]:
    #print(d)
    plt.plot(derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']])
    plt.show()
    #d = dict_comm_prices[d][0]
    #print(d)
"""

"\nfor d in [d for d in dict_comm_prices]:\n    #print(d)\n    plt.plot(derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']])\n    plt.show()\n    #d = dict_comm_prices[d][0]\n    #print(d)\n"

In [39]:
"""
#d= [d for d in dict_comm_prices][0]
comm_tvv_df_pvt = pd.DataFrame()

for d in [d for d in dict_comm_prices]:
    #print(d)    
    test = derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']]
        
    #if(np.sum(dict_comm_p
    if(abs(np.sum(test))[0]>0):
        if((len(test.dropna()))<int(0.5*test.shape[0])):
            temp = derive_price_supply_trends([d,dict_comm_prices[d][['Adj Close']],nyse_dates_X2])[['risk_50td_trend']]
        else:
            temp = derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']]            
    else:
        temp = derive_price_supply_trends([d,dict_comm_prices[d][['Adj Close']],nyse_dates_X2])[['risk_50td_trend']]
    
    temp.columns = [d]
    
    temp = temp.asfreq('D').interpolate().asfreq('W-'+end.strftime('%a'))
        
    comm_tvv_df_pvt = pd.concat([comm_tvv_df_pvt,temp],axis=1)
"""

"\n#d= [d for d in dict_comm_prices][0]\ncomm_tvv_df_pvt = pd.DataFrame()\n\nfor d in [d for d in dict_comm_prices]:\n    #print(d)    \n    test = derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']]\n        \n    #if(np.sum(dict_comm_p\n    if(abs(np.sum(test))[0]>0):\n        if((len(test.dropna()))<int(0.5*test.shape[0])):\n            temp = derive_price_supply_trends([d,dict_comm_prices[d][['Adj Close']],nyse_dates_X2])[['risk_50td_trend']]\n        else:\n            temp = derive_tvv([d,dict_comm_prices[d],nyse_dates_X2])[['50td_tvf_vel']]            \n    else:\n        temp = derive_price_supply_trends([d,dict_comm_prices[d][['Adj Close']],nyse_dates_X2])[['risk_50td_trend']]\n    \n    temp.columns = [d]\n    \n    temp = temp.asfreq('D').interpolate().asfreq('W-'+end.strftime('%a'))\n        \n    comm_tvv_df_pvt = pd.concat([comm_tvv_df_pvt,temp],axis=1)\n"

In [40]:
#comm_tvv_df_pvt = pd.pivot_table(comm_tvv_df, values=['50td_tvf_vel'], index=comm_tvv_df.index,columns=['Symbol'])
#comm_tvv_df_pvt.columns = comm_tvv_df_pvt.columns.droplevel(0)
"""
comm_tvv_df_10_yr_pvt = comm_tvv_df_pvt.dropna(axis=1, thresh = int(0.95*comm_tvv_df_pvt.shape[0]))
comm_tvv_df_5_yr_pvt = comm_tvv_df_pvt.tail(int(len(comm_tvv_df_10_yr_pvt)/2))
comm_tvv_df_5_yr_pvt = comm_tvv_df_5_yr_pvt.dropna(axis=1, thresh = int(0.95*comm_tvv_df_5_yr_pvt.shape[0]))
"""

'\ncomm_tvv_df_10_yr_pvt = comm_tvv_df_pvt.dropna(axis=1, thresh = int(0.95*comm_tvv_df_pvt.shape[0]))\ncomm_tvv_df_5_yr_pvt = comm_tvv_df_pvt.tail(int(len(comm_tvv_df_10_yr_pvt)/2))\ncomm_tvv_df_5_yr_pvt = comm_tvv_df_5_yr_pvt.dropna(axis=1, thresh = int(0.95*comm_tvv_df_5_yr_pvt.shape[0]))\n'

In [41]:
"""
comm_tvv_df_10_yr_pvt_corr = comm_tvv_df_10_yr_pvt.asfreq('D').interpolate(method='time').asfreq('W-'+comm_tvv_df_10_yr_pvt.index[-1].strftime('%a')).corr()
comm_tvv_df_5_yr_pvt_corr = comm_tvv_df_5_yr_pvt.asfreq('D').interpolate(method='time').asfreq('W-'+comm_tvv_df_5_yr_pvt.index[-1].strftime('%a')).corr()
"""

"\ncomm_tvv_df_10_yr_pvt_corr = comm_tvv_df_10_yr_pvt.asfreq('D').interpolate(method='time').asfreq('W-'+comm_tvv_df_10_yr_pvt.index[-1].strftime('%a')).corr()\ncomm_tvv_df_5_yr_pvt_corr = comm_tvv_df_5_yr_pvt.asfreq('D').interpolate(method='time').asfreq('W-'+comm_tvv_df_5_yr_pvt.index[-1].strftime('%a')).corr()\n"

In [42]:
writer = pd.ExcelWriter('..\\reports\\'+end.strftime('%Y-%m-%d')+'_correlations.xlsx', engine='xlsxwriter')



In [43]:

#price_comm_df_10_yr_pvt_corr.to_excel(writer, sheet_name='price_comm_df_10_yr_pvt_corr')
#price_comm_df_5_yr_pvt_corr.to_excel(writer, sheet_name='price_comm_df_5_yr_pvt_corr')

#future_commodities_price_pvt = pd.pivot_table(commodity_prices, values=['Adj Close'], index=commodity_prices.index,columns=['Symbol'])
#future_commodities_price_pvt.columns = future_commodities_price_pvt.columns.droplevel(0)

In [44]:


#METALS CORRELATION MATRIX doing the same thing with (1) 50 day price change and (2) 50 day TVV.  Please run them all against each other to complete the METALS correlation matrix.  Run a 5 year and 10 year Matrix

#future_commodities_price_pvt_corr = future_commodities_price_pvt.corr().replace([np.NaN],0)

#future_commodities_50td_tvv_pvt.columns = full_commodities_df_pvt.columns.droplevel(0)

#'MVMTD027MNFRBDAL'
#check_corr_list = ['^GSPC','DGS3MO','DGS2','DGS10']
#check_corr_list = [dict_metals[d] for d in dict_metals]

"""
for c_c in check_corr_list:
    #inside = future_commodities_50td_tvv_pvt_corr.sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    inside = comm_tvv_df_10_yr_pvt[dict_metals[d] for d in dict_metals].corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    plt.figure(figsize=(24,15))
    sns.heatmap(inside,cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'comm_tvv_df_10_yr_pvt_corr.png', dpi=300, format='png', bbox_inches='tight')
    plt.show()
    
"""


#price_comm_df_10_yr_pvt_corr[price_comm_df_10_yr_pvt_corr.columns.intersection([dict_metals[d] for d in dict_metals])].corr().to_excel(writer, sheet_name='price_comm_metals_10_yr_corr')
#price_comm_df_5_yr_pvt_corr[price_comm_df_5_yr_pvt_corr.columns.intersection([dict_metals[d] for d in dict_metals])].corr().to_excel(writer, sheet_name='price_comm_metals_5_yr_corr')

'\nfor c_c in check_corr_list:\n    #inside = future_commodities_50td_tvv_pvt_corr.sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)\n    inside = comm_tvv_df_10_yr_pvt[dict_metals[d] for d in dict_metals].corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)\n    plt.figure(figsize=(24,15))\n    sns.heatmap(inside,cmap="RdYlGn",annot=True)\n    plt.savefig(\'..\\reports\\figures\\\'+end.strftime(\'%Y-%m-%d\')+\'-\'+c_c+\'comm_tvv_df_10_yr_pvt_corr.png\', dpi=300, format=\'png\', bbox_inches=\'tight\')\n    plt.show()\n    \n'

In [45]:

merged_dict = dict_treasury | dict_currencies_futures | dict_energy | dict_foods | dict_indexes_w_futures | dict_materials_futures | dict_metals

templist = [merged_dict[d] for d in merged_dict]

A = templist

In [46]:
#price_comm_df_10_yr_pvt_corr[set[price_comm_df_10_yr_pvt_corr.columns].intersection(set(currencies))].corr().to_excel(writer, sheet_name='price_comm_curr_10_yr_corr')
#price_comm_df_5_yr_pvt_corr[set[price_comm_df_5_yr_pvt_corr.columns].intersection(set(currencies))].corr().to_excel(writer, sheet_name='price_comm_curr_5_yr_corr')

In [47]:
#price_comm_df_10_yr_pvt_corr[price_comm_df_10_yr_pvt_corr.columns.intersection([*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures]])].corr().to_excel(writer, sheet_name='price_comm_curr_10_yr_corr')


B = price_comm_df_10_yr_pvt_corr.columns
common = sorted(set(A).intersection(B) ,key=lambda x:A.index(x))

reordered_10_yr = price_comm_df_10_yr_pvt_corr[common]
reordered_10_yr.columns = [dict((v, k) for k, v in merged_dict.items())[f] for f in common]
reordered_10_yr.loc[common].T.to_excel(writer, sheet_name='prc_comm_10_yr_wt_corr')

In [48]:
#price_comm_df_5_yr_pvt_corr[price_comm_df_5_yr_pvt_corr.columns.intersection([*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures]])].corr().to_excel(writer, sheet_name='price_comm_curr_5_yr_corr')


B = price_comm_df_5_yr_pvt_corr.columns
common = sorted(set(A).intersection(B) ,key=lambda x:A.index(x))

reordered_5_yr = price_comm_df_5_yr_pvt_corr[common]
reordered_5_yr.columns = [dict((v, k) for k, v in merged_dict.items())[f] for f in common]
reordered_5_yr.loc[common].T.to_excel(writer, sheet_name='prc_comm_5_yr_wt_corr')

In [49]:
#price_comm_df_2_yr_pvt_corr[price_comm_df_2_yr_pvt_corr.columns.intersection([*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures]])].corr().to_excel(writer, sheet_name='price_comm_curr_2_yr_corr')


B = price_comm_df_2_yr_pvt_corr.columns
common = sorted(set(A).intersection(B) ,key=lambda x:A.index(x))

reordered_2_yr = price_comm_df_2_yr_pvt_corr[common]
reordered_2_yr.columns = [dict((v, k) for k, v in merged_dict.items())[f] for f in common]
reordered_2_yr.loc[common].T.to_excel(writer, sheet_name='prc_comm_2_yr_wt_corr')

In [50]:
a = price_comm_df_10_yr_pvt_corr[price_comm_df_10_yr_pvt_corr.columns.intersection([*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures]])].corr()
b = price_comm_df_5_yr_pvt_corr[price_comm_df_5_yr_pvt_corr.columns.intersection([*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures]])].corr()*2
c = price_comm_df_2_yr_pvt_corr[price_comm_df_2_yr_pvt_corr.columns.intersection([*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures]])].corr()

weighted_10y_corr = ((a+b+c)/4)

A = templist
B = weighted_10y_corr.columns
common = sorted(set(A).intersection(B) ,key=lambda x:A.index(x))

reordered_10_yr = weighted_10y_corr[common]
reordered_10_yr.columns = [dict((v, k) for k, v in merged_dict.items())[f] for f in common]
reordered_10_yr.loc[common].T.to_excel(writer, sheet_name='prc_comm_wgt_10_yr_wt_corr')

In [51]:
#price_comm_df_10_yr_pvt_corr[price_comm_df_10_yr_pvt_corr.columns.intersection([dict_foods[d] for d in dict_foods])].corr().to_excel(writer, sheet_name='price_comm_foods_10_yr_corr')
#price_comm_df_5_yr_pvt_corr[price_comm_df_5_yr_pvt_corr.columns.intersection([dict_foods[d] for d in dict_foods])].corr().to_excel(writer, sheet_name='price_comm_foods_5_yr_corr')

In [52]:
#price_comm_df_10_yr_pvt_corr[price_comm_df_10_yr_pvt_corr.columns.intersection([dict_energy[d] for d in dict_energy])].corr().to_excel(writer, sheet_name='price_comm_energy_10_yr_corr')
#price_comm_df_5_yr_pvt_corr[price_comm_df_5_yr_pvt_corr.columns.intersection([dict_energy[d] for d in dict_energy])].corr().to_excel(writer, sheet_name='price_comm_energy_5_yr_corr')

In [53]:
writer.close()

In [54]:
"""
price_comm_df_10_yr_pvt_corr.to_excel(writer, sheet_name='price_comm_df_10_yr_pvt_corr')
price_comm_df_5_yr_pvt_corr.to_excel(writer, sheet_name='comm_t')
pd.DataFrame(sp400).to_excel(writer, sheet_name='SP400')
pd.DataFrame(market_data).to_excel(writer, sheet_name='Stock_Fundamentals')
completed_fred_pvt.to_excel(writer, sheet_name='completed_fred_pvt')
completed_bonds.to_excel(writer, sheet_name='completed_bonds')
completed_bonds_pvt.to_excel(writer, sheet_name='completed_bonds_pvt')
pd.DataFrame(pd.DataFrame(sectors,sector_names)).to_excel(writer, sheet_name='vanguard_sectors')
pd.DataFrame(pd.DataFrame(indexes,index_names)).to_excel(writer, sheet_name='spdr_indexes')
screener_metrics.to_excel(writer, sheet_name='Screener')

writer.save()

wb = openpyxl.load_workbook('SP1500_screener_multiple.xlsx')
#openpyxl.Workbook()
new_range = openpyxl.workbook.defined_name.DefinedName('Prices', attr_text='SP1500_Sectors_Indexes_Prices!$A$1:$H$'+str(len(prices_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SP1500', attr_text='SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Vanguard_Sectors', attr_text='vanguard_sectors!$B$1:$C$'+str(len(sectors)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SPDR_Indexes', attr_text='spdr_indexes!$B$1:$C$'+str(len(indexes)+1))
wb.defined_names.append(new_range)


#wb.defined_names.append(new_range)
wb.save('SP1500_screener_multiple.xlsx')
wb.close()
#named_range = NamedRange("test_nr", [(new_sheet, "A1")])
#wb.add_named_range(new_range)
wb.close()

"""
"""
workbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')
workbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))
workbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
workbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
workbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))

workbook.close()
"""

"\nworkbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')\nworkbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))\nworkbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))\nworkbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))\nworkbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))\n\nworkbook.close()\n"

In [55]:
#TVV

"""
tvv_corr_set = np.unique([*[dict_treasury[d] for d in dict_treasury],*[d for d in other_commodities_list],*[d for d in indexes_futures]])
"""

'\ntvv_corr_set = np.unique([*[dict_treasury[d] for d in dict_treasury],*[d for d in other_commodities_list],*[d for d in indexes_futures]])\n'

In [56]:
"""
list1_as_set = set(tvv_corr_set)

intersection = comm_tvv_df_10_yr_pvt_corr.columns.intersection(list1_as_set)
intersection_as_list = list(intersection)

comm_tvv_df_10_yr_pvt_corr.loc[intersection_as_list][intersection_as_list].to_excel(writer, sheet_name='comm_tvv_10_yr_corr')

list1_as_set = set(tvv_corr_set)
intersection = comm_tvv_df_5_yr_pvt_corr.columns.intersection(list1_as_set)
intersection_as_list = list(intersection)

comm_tvv_df_5_yr_pvt_corr.loc[intersection_as_list][intersection_as_list].to_excel(writer, sheet_name='comm_tvv_5_yr_corr')

"""

"\nlist1_as_set = set(tvv_corr_set)\n\nintersection = comm_tvv_df_10_yr_pvt_corr.columns.intersection(list1_as_set)\nintersection_as_list = list(intersection)\n\ncomm_tvv_df_10_yr_pvt_corr.loc[intersection_as_list][intersection_as_list].to_excel(writer, sheet_name='comm_tvv_10_yr_corr')\n\nlist1_as_set = set(tvv_corr_set)\nintersection = comm_tvv_df_5_yr_pvt_corr.columns.intersection(list1_as_set)\nintersection_as_list = list(intersection)\n\ncomm_tvv_df_5_yr_pvt_corr.loc[intersection_as_list][intersection_as_list].to_excel(writer, sheet_name='comm_tvv_5_yr_corr')\n\n"

In [57]:
"""
list1_as_set = set(tvv_corr_set)
intersection = comm_tvv_df_10_yr_pvt_corr.columns.intersection(list1_as_set)
intersection_as_list = list(intersection)


for d in [d for d in dict_treasury]:
    #print(d)
    d = dict_treasury[d]
    print(d)
    
    inside_10 = comm_tvv_df_10_yr_pvt_corr[intersection_as_list].sort_values(by=d,ascending=False,axis=0).sort_values(by=d,ascending=False,axis=1)
    inside_5 = comm_tvv_df_5_yr_pvt_corr[intersection_as_list].sort_values(by=d,ascending=False,axis=0).sort_values(by=d,ascending=False,axis=1)
    
    inside_10[intersection_as_list].to_excel(writer, sheet_name=d+'_comm_tvv_10_yr_corr')
    inside_5[intersection_as_list].to_excel(writer, sheet_name=d+'_comm_tvv_5_yr_corr')

    #print('10 yr')
    #plt.figure(figsize=(24,15))
    #sns.heatmap(inside_10,cmap="RdYlGn",annot=True)
    #plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+d+'price_comm_df_10_yr_pvt_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()
    #print('5 yr')
    #plt.figure(figsize=(24,15))
    #sns.heatmap(inside_5,cmap="RdYlGn",annot=True)
    #plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+d+'price_comm_df_5_yr_pvt_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()
    
"""

"""    
for 
#price
#price_corr_set = 
[dict_energy[d] for d in dict_energy]
[dict_metals[d] for d in dict_metals]
[currencies[d] for d in currencies]

[dict_foods[d] for d in dict_foods]
"""

'    \nfor \n#price\n#price_corr_set = \n[dict_energy[d] for d in dict_energy]\n[dict_metals[d] for d in dict_metals]\n[currencies[d] for d in currencies]\n\n[dict_foods[d] for d in dict_foods]\n'

In [58]:

tbill_3mo_df = Fred_Data(['DGS3MO',start_X2,end,nyse_dates_X2])[['Value']]
tbill_3mo_df.columns = ['DGS3MO']

tbill_2y_df = Fred_Data(['DGS2',start_X2,end,nyse_dates_X2])[['Value']]
tbill_2y_df.columns = ['DGS2']

tbill_10y_df = Fred_Data(['DGS10',start_X2,end,nyse_dates_X2])[['Value']]
tbill_10y_df.columns = ['DGS10']


In [59]:
future_commodities_n_w_index_pvt_w_Fred = price_comm_df_10_yr_pvt[price_comm_df_10_yr_pvt.columns.intersection([*[dict_treasury[d] for d in dict_treasury], *[dict_metals[d] for d in dict_metals], *[dict_currencies_futures[d] for d in dict_currencies_futures], *[dict_energy[d] for d in dict_energy],*[dict_foods[d] for d in dict_foods], *[dict_indexes_w_futures[d] for d in dict_indexes_w_futures], *[dict_materials_futures[d] for d in dict_materials_futures]])]

In [60]:
#future_commodities_n_w_index_pvt_w_Fred = pd.concat([tbill_3mo_df,tbill_2y_df,tbill_10y_df,],axis=1)

future_commodities_n_w_index_pvt_w_Fred.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'_future_commodities_n_w_index_pvt_w_Fred.csv')
future_commodities_n_w_index_pvt_w_Fred.dropna(axis=1, thresh = int(.95*future_commodities_n_w_index_pvt_w_Fred.shape[0]), inplace=True)


In [61]:
#for c_c in check_corr_list:
    #future_commodities_50td_tvv_pvt = pd.concat([future_commodities_50td_tvv_pvt,future_commodities_n_w_index_pvt_w_Fred[[c_c]]],axis=1)


In [62]:
"""
future_commodities_50td_tvv_pvt_corr = future_commodities_50td_tvv_pvt.corr().replace([np.NaN],0)

#future_commodities_50td_tvv_pvt.columns = full_commodities_df_pvt.columns.droplevel(0)

#'MVMTD027MNFRBDAL'
#check_corr_list = ['^GSPC','DGS3MO','DGS2','DGS10']
check_corr_list = [dict_treasury[d] for d in dict_treasury]

for c_c in check_corr_list:
    #inside = future_commodities_50td_tvv_pvt_corr.sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    inside = comm_tvv_df_10_yr_pvt.corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    plt.figure(figsize=(24,15))
    sns.heatmap(inside,cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'comm_tvv_df_10_yr_pvt_corr.png', dpi=300, format='png', bbox_inches='tight')
    plt.show()
"""

'\nfuture_commodities_50td_tvv_pvt_corr = future_commodities_50td_tvv_pvt.corr().replace([np.NaN],0)\n\n#future_commodities_50td_tvv_pvt.columns = full_commodities_df_pvt.columns.droplevel(0)\n\n#\'MVMTD027MNFRBDAL\'\n#check_corr_list = [\'^GSPC\',\'DGS3MO\',\'DGS2\',\'DGS10\']\ncheck_corr_list = [dict_treasury[d] for d in dict_treasury]\n\nfor c_c in check_corr_list:\n    #inside = future_commodities_50td_tvv_pvt_corr.sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)\n    inside = comm_tvv_df_10_yr_pvt.corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)\n    plt.figure(figsize=(24,15))\n    sns.heatmap(inside,cmap="RdYlGn",annot=True)\n    plt.savefig(\'..\\reports\\figures\\\'+end.strftime(\'%Y-%m-%d\')+\'-\'+c_c+\'comm_tvv_df_10_yr_pvt_corr.png\', dpi=300, format=\'png\', bbox_inches=\'tight\')\n    plt.show()\n'

In [63]:
"""

check_corr_list = ['^GSPC','DGS3MO','DGS2','DGS10']

for c_c in check_corr_list:

    data = future_commodities_n_w_index_pvt_w_Fred[[c_c]]
    dates = [pd.to_datetime(d).value for d in data.index]

    dates = np.array(dates)-np.array(dates[0])/10**11

    int_date_data = pd.concat([pd.DataFrame(dates),pd.DataFrame(data[[c_c]].values)],axis=1)
    int_date_data.columns = ['Date',c_c]

    windows = split_sequences(int_date_data.values,n_trend)
    windows = pd.DataFrame(windows).head(-(n_trend-1)).values.ravel()        

    trend_t = [*[trend_tau([pd.DataFrame(d).iloc[:,0].values,pd.DataFrame(d).iloc[:,1].values]) for d in windows]]
    sp_w_trend = pd.concat([pd.DataFrame(data[c_c].values),pd.DataFrame(np.append(np.repeat(np.nan,len(data)-len([t[0] for t in trend_t])),[t[0] for t in trend_t])),pd.DataFrame(np.append(np.repeat(np.nan,len(data)-len([t[0] for t in trend_t])),[t[0] for t in trend_t]))],axis=1)
    sp_w_trend.columns = [c_c,'trending','p-value']
    sp_w_trend.index = data.index
    trending = sp_w_trend[['trending']]>0  
    #trending = pd.DataFrame((future_commodities_n_w_index_pvt_w_Fred[c_c]-future_commodities_n_w_index_pvt_w_Fred[c_c].shift(n_trend))>0)

    trending.columns = ['trending']
    trending_df = pd.concat([future_commodities_n_w_index_pvt_w_Fred,trending],axis=1)

    future_commodities_trending = trending_df[trending_df['trending']==True]
    future_commodities_not_trending = trending_df[trending_df['trending']!=True]

    sorted_by_c_future_commodities_corr = future_commodities_n_w_index_pvt_w_Fred.corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    sorted_by_c_future_commodities_corr.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_sorted_by_c_future_commodities_corr.csv')

    future_commodities_trending_corr = future_commodities_trending.corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    sorted_by_c_future_commodities_corr.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_future_commodities_trending_corr.csv')

    future_commodities_not_trending_corr = future_commodities_not_trending.corr().sort_values(by=c_c,ascending=False,axis=0).sort_values(by=c_c,ascending=False,axis=1)
    future_commodities_not_trending_corr.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_future_commodities_not_trending_corr.csv')

    plt.figure(figsize=(24,15))
    sns.heatmap(future_commodities_n_w_index_pvt_w_Fred[sorted_by_c_future_commodities_corr.index].corr(),cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_sorted_by_c_future_commodities_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()

    fig, ax1 = plt.subplots( figsize=(15,6) )

    ax2 = ax1.twinx()

    ax1.plot(trending_df[[c_c]])
    ax2.plot(trending_df[['trending']])
    ax2.fill_between(trending_df.index, 0, 1, where=trending_df['trending'] == True,facecolor='blue', alpha=0.5)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_trending.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()

    print("correlations when trending")
    print(len(future_commodities_trending))

    sorted_future_commodities_trending = future_commodities_trending.corr()[[c_c]].sort_values(by=c_c,ascending=False)

    plt.figure(figsize=(24,15))
    sns.heatmap(future_commodities_trending[sorted_future_commodities_trending.index].corr(),cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_sorted_by_c_future_commodities_trending_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()

    print("correlations when not trending")
    print(len(future_commodities_not_trending))

    sorted_future_commodities_not_trending = future_commodities_not_trending.corr()[[c_c]].sort_values(by=c_c,ascending=False)

    plt.figure(figsize=(24,15))
    sns.heatmap(future_commodities_not_trending[sorted_future_commodities_not_trending.index].corr(),cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_sorted_by_c_future_commodities_not_trending_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()

    plt.figure(figsize=(4,5))
    sns.heatmap((sorted_future_commodities_not_trending-sorted_future_commodities_trending).sort_values(by=[c_c],ascending=False),cmap="RdYlGn",annot=True)
    plt.savefig('..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_sorted_by_c_future_commodities_trending-not_trending_corr.png', dpi=300, format='png', bbox_inches='tight')
    #plt.show()

    n = len(future_commodities_not_trending)

    dist = scipy.stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)
    p_not_trending = pd.DataFrame(2*dist.cdf(-abs(future_commodities_not_trending[sorted_future_commodities_not_trending.index].corr())),columns=future_commodities_not_trending.columns,index=sorted_future_commodities_not_trending.index)

    n = len(future_commodities_trending)

    dist = scipy.stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)
    p_trending = pd.DataFrame(2*dist.cdf(-abs(future_commodities_trending[sorted_future_commodities_trending.index].corr())),columns=future_commodities_trending.columns,index=sorted_future_commodities_trending.index)
    #p_trending = pd.DataFrame([2*dist.cdf(-abs(future_commodities_trending[sorted_future_commodities_trending.index].corr()))],columns=sorted_future_commodities_trending.columns,index=sorted_future_commodities_trending.index)


    temp_all_significant = pd.DataFrame((pd.DataFrame(p_not_trending)<.05)*(pd.DataFrame(p_trending)<.05)*(future_commodities_not_trending[sorted_future_commodities_not_trending.index].corr()*future_commodities_trending[sorted_future_commodities_trending.index].corr())>0)
    temp_all_significant_inverted = pd.DataFrame((pd.DataFrame(p_not_trending)<.05)*(pd.DataFrame(p_trending)<.05)*(future_commodities_not_trending[sorted_future_commodities_not_trending.index].corr()*future_commodities_trending[sorted_future_commodities_trending.index].corr())<0)
    temp_trend_significant = pd.DataFrame(pd.DataFrame(p_trending)<.05)
    temp_not_trend_significant = pd.DataFrame(pd.DataFrame(p_not_trending)<.05)

    combined = pd.concat([temp_trend_significant.loc[[c_c]].T,temp_not_trend_significant.loc[[c_c]].T,temp_all_significant.loc[[c_c]].T,temp_all_significant_inverted.loc[[c_c]].T],axis=1)
    combined.columns = ['trend','not_trend','both','inverted']
    df_styled = combined.style.applymap(highlight_cells)
    #display(df_styled)
    #with HiddenPrints():
    dfi.export(df_styled, '..\\reports\\figures\\'+end.strftime('%Y-%m-%d')+'-'+c_c+'_corr_styled.png'
"""

'\n\ncheck_corr_list = [\'^GSPC\',\'DGS3MO\',\'DGS2\',\'DGS10\']\n\nfor c_c in check_corr_list:\n\n    data = future_commodities_n_w_index_pvt_w_Fred[[c_c]]\n    dates = [pd.to_datetime(d).value for d in data.index]\n\n    dates = np.array(dates)-np.array(dates[0])/10**11\n\n    int_date_data = pd.concat([pd.DataFrame(dates),pd.DataFrame(data[[c_c]].values)],axis=1)\n    int_date_data.columns = [\'Date\',c_c]\n\n    windows = split_sequences(int_date_data.values,n_trend)\n    windows = pd.DataFrame(windows).head(-(n_trend-1)).values.ravel()        \n\n    trend_t = [*[trend_tau([pd.DataFrame(d).iloc[:,0].values,pd.DataFrame(d).iloc[:,1].values]) for d in windows]]\n    sp_w_trend = pd.concat([pd.DataFrame(data[c_c].values),pd.DataFrame(np.append(np.repeat(np.nan,len(data)-len([t[0] for t in trend_t])),[t[0] for t in trend_t])),pd.DataFrame(np.append(np.repeat(np.nan,len(data)-len([t[0] for t in trend_t])),[t[0] for t in trend_t]))],axis=1)\n    sp_w_trend.columns = [c_c,\'trending\',\'

In [64]:
#(pd.DataFrame(p_not_trending)<.05)
#(pd.DataFrame(p_trending)<.05)

In [65]:
#test

dates = []
for d in pd.read_csv('..\\data\\raw\\sample.csv')['Date']:
    dates.append(dt.datetime.strptime(d, '%d-%b-%y'))

temp = pd.read_csv('..\\data\\raw\\sample.csv',thousands=",")
temp.index = dates
temp = temp.iloc[:,1:]
temp.columns = ['Open','High','Low','Adj Close','Volume']
#temp['Close'] = temp['Close'].astype(float)
temp.sort_index(inplace=True)
temp['Symbol'] = 'Test'

temp_new = derive_tvv(['temp',temp,nyse_dates]).sort_index(ascending=False)
temp_new.sort_index(inplace=True)
#display(temp_new)
#display(temp_new[['buy_volume_pct', 'sell_volume_pct', 'daily_buy_vol', 'daily_sell_vol', 'net_daily_vol', '20td_trend_vol', '20td_tot_daily_vol','20td_trend_vol_factor', '20td_tvf_rng_div_2', '20td_tvf_vel',  '50td_trend_vol', '50td_tot_daily_vol','50td_trend_vol_factor', '50td_tvf_rng_div_4', '50td_tvf_vel']])
#display(calculateMetrics(temp_new))
#


In [66]:
npa = []
for i in range(0,len([*SP1500,*indexes,*sectors])):
    npa.append([[*SP1500,*indexes,*sectors][i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
print("npa length:getStock(stocks)",len(npa))

# = getStock(npa[0])

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()
future = client.map(getStock, npa)
results_prices = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_prices.append("error")
    else:
        results_prices.append(f.result()) 
client.close()


npa length:getStock(stocks) 1520


C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\distributed\client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.10.6 | None      | None    |
| lz4     | 4.0.2  | 3.1.10    | 3.1.10  |
| numpy   | 1.23.1 | 1.22.4    | 1.22.4  |
| tornado | 6.2    | 6.1       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [67]:
npa = []
for i in range(0,len([*SP1500,*indexes,*sectors])):
    npa.append([[*SP1500,*indexes,*sectors][i]])
    
print("npa length:getTickerData",len(npa))

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(getTickerData, npa)

results_ticker_data = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_data.append("error")
    else:
        results_ticker_data.append(f.result()) 
#client.restart()
client.close()


npa length:getTickerData 1520


In [68]:
print(results_ticker_data[0][1].info)
print(results_ticker_data[0][1].quarterly_balance_sheet)
print(results_ticker_data[0][1].quarterly_cashflow)
#print(results_ticker_data[0][1].quarterly_dividends)
print(results_ticker_data[0][1].quarterly_earnings)
print(results_ticker_data[0][1].quarterly_financials)


{'zip': '45202', 'sector': 'Consumer Defensive', 'longBusinessSummary': 'The Procter & Gamble Company provides branded consumer packaged goods worldwide. It operates through five segments: Beauty; Grooming; Health Care; Fabric & Home Care; and Baby, Feminine & Family Care. The Beauty segment offers conditioners, shampoos, styling aids, and treatments under the Head & Shoulders, Herbal Essences, Pantene, and Rejoice brands; and antiperspirants and deodorants, personal cleansing, and skin care products under the Olay, Old Spice, Safeguard, Secret, and SK-II brands. The Grooming segment provides shave care products and appliances under the Braun, Gillette, and Venus brand names. The Health Care segment offers toothbrushes, toothpastes, and other oral care products under the brand names of Crest and Oral-B; and gastrointestinal, rapid diagnostics, respiratory, vitamins/minerals/supplements, pain relief, and other personal health care products under the Metamucil, Neurobion, Pepto-Bismol, a

In [69]:
results_ticker_data[0][1].info

{'zip': '45202',
 'sector': 'Consumer Defensive',
 'longBusinessSummary': 'The Procter & Gamble Company provides branded consumer packaged goods worldwide. It operates through five segments: Beauty; Grooming; Health Care; Fabric & Home Care; and Baby, Feminine & Family Care. The Beauty segment offers conditioners, shampoos, styling aids, and treatments under the Head & Shoulders, Herbal Essences, Pantene, and Rejoice brands; and antiperspirants and deodorants, personal cleansing, and skin care products under the Olay, Old Spice, Safeguard, Secret, and SK-II brands. The Grooming segment provides shave care products and appliances under the Braun, Gillette, and Venus brand names. The Health Care segment offers toothbrushes, toothpastes, and other oral care products under the brand names of Crest and Oral-B; and gastrointestinal, rapid diagnostics, respiratory, vitamins/minerals/supplements, pain relief, and other personal health care products under the Metamucil, Neurobion, Pepto-Bismol,

In [70]:
npa = []
results_ticker_names = [r[0] for r in results_ticker_data]
for n in range(0,len(results_ticker_names),1):
    npa.append([results_ticker_names[n],results_ticker_data[n][1]])

In [71]:

print("npa length:extractTickerValues",len(npa))  


npa length:extractTickerValues 1520


In [72]:

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(extractTickerValues, npa)

results_ticker_values = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_values.append("error")
    else:
        results_ticker_values.append(f.result()) 
client.close()

"""
trackers = ['beta','bookValue','currentRatio','debtToEquity','dividendRate','dividendYield','earningsGrowth','ebitda','ebitdaMargins','enterpriseValue','enterpriseToEbitda','enterpriseToRevenue','floatShares','forwardPE','freeCashflow','grossMargins','grossProfits','heldPercentInsiders','heldPercentInstitutions','impliedSharesOutstanding','industry','marketCap','operatingCashflow','operatingMargins','payoutRatio','pegRatio','priceToBook','priceToSalesTrailing12Months','profitMargins','quickRatio','returnOnAssets','returnOnEquity','revenueGrowth','revenuePerShare','sector','sharesOutstanding','sharesShort','sharesPercentSharesOut','sharesShortPreviousMonthDate','shortRatio','totalAssets','totalCash','totalCashPerShare','totalDebt','totalRevenue','trailingPE']

for t in trackers:
    if(np.sum(np.where(subset.columns==t))==0):
        values = pd.concat([values,pd.DataFrame('error',index=[n_],columns=[t])],axis=1)
    else:
        values = pd.concat([values,pd.DataFrame(subset[t].values[0],index=[n_],columns=[t])],axis=1)
"""

C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\distributed\client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.10.6 | None      | None    |
| lz4     | 4.0.2  | 3.1.10    | 3.1.10  |
| numpy   | 1.23.1 | 1.22.4    | 1.22.4  |
| tornado | 6.2    | 6.1       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


"\ntrackers = ['beta','bookValue','currentRatio','debtToEquity','dividendRate','dividendYield','earningsGrowth','ebitda','ebitdaMargins','enterpriseValue','enterpriseToEbitda','enterpriseToRevenue','floatShares','forwardPE','freeCashflow','grossMargins','grossProfits','heldPercentInsiders','heldPercentInstitutions','impliedSharesOutstanding','industry','marketCap','operatingCashflow','operatingMargins','payoutRatio','pegRatio','priceToBook','priceToSalesTrailing12Months','profitMargins','quickRatio','returnOnAssets','returnOnEquity','revenueGrowth','revenuePerShare','sector','sharesOutstanding','sharesShort','sharesPercentSharesOut','sharesShortPreviousMonthDate','shortRatio','totalAssets','totalCash','totalCashPerShare','totalDebt','totalRevenue','trailingPE']\n\nfor t in trackers:\n    if(np.sum(np.where(subset.columns==t))==0):\n        values = pd.concat([values,pd.DataFrame('error',index=[n_],columns=[t])],axis=1)\n    else:\n        values = pd.concat([values,pd.DataFrame(subset[

In [73]:
 
ticker_values = pd.DataFrame()

pickle_dumps = []

for r in results_ticker_values:
    if(r=='error'):
        pass
        #pd.DataFrame(np.repeat('error', len(trackers), axis = None),index=)
    else:                
        #print(r[1])
        subset = pd.DataFrame(r[1])
        subset.index = [r[0]]
        #print(subset)
        ticker_values = pd.concat([ticker_values,subset],axis=0)
        pickle_dumps.append([r[0],r[2]])
        #ticker_values = pd.concat([ticker_values,pd.DataFrame(r[1],index=[r[0]])],axis=0)

pd.DataFrame(ticker_values)

pickle.dump(pickle_dumps, open('..\\data\\interim\\fundamental.pkl', 'wb'))

In [74]:
"""
missing_ticker_values = list(set([*SP1500,*indexes,*sectors]).difference([n for n in pd.DataFrame(ticker_values).index]))
print(missing_ticker_values)

for r in range(0,len(missing_ticker_values)):
    r_ = missing_ticker_values[r]
    temp = pd.DataFrame(np.array([r_,*np.repeat('missing',len(pd.DataFrame(ticker_values.columns)))])).T.set_index(0)
    temp.columns = ticker_values.columns
    #.set_index(0).iloc[[0]].columns))])
    ticker_values = pd.concat([ticker_values,temp],axis=0)
"""


"\nmissing_ticker_values = list(set([*SP1500,*indexes,*sectors]).difference([n for n in pd.DataFrame(ticker_values).index]))\nprint(missing_ticker_values)\n\nfor r in range(0,len(missing_ticker_values)):\n    r_ = missing_ticker_values[r]\n    temp = pd.DataFrame(np.array([r_,*np.repeat('missing',len(pd.DataFrame(ticker_values.columns)))])).T.set_index(0)\n    temp.columns = ticker_values.columns\n    #.set_index(0).iloc[[0]].columns))])\n    ticker_values = pd.concat([ticker_values,temp],axis=0)\n"

In [75]:
market_data = pd.concat([ticker_values])

In [76]:
names_prices = [rp[0] for rp in results_prices]

#def process_df(data_):
npa = []    
for n in range(0,len(names_prices)):
    #print(names_prices[n])
    n_ = names_prices[n]
    npa.append([n_,results_prices[n][1],nyse_dates])
    
print("npa length:derive_tvv",len(npa))

prices_df = pd.DataFrame()

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(derive_tvv, npa)
#results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        #results_sectors.append("error")
        pass
    else:
        #results_sectors.append(f.result())
        prices_df = pd.concat([f.result(),prices_df],axis=0)
client.close()

npa length:derive_tvv 1520


In [77]:

npa = []
for s in names_prices:
    #print(s)
    subset = prices_df.iloc[np.where(prices_df['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
        
print("npa length:calculateMetrics",len(npa))

screenerMetrics = []

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(calculateMetrics, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        screenerMetrics.append("error")
    else:
        screenerMetrics.append(f.result()) 
client.close()



npa length:calculateMetrics 1519


In [78]:
screener_metrics = pd.concat(screenerMetrics)

In [79]:

missing_screener_metrics = list(set([*SP1500,*indexes,*sectors]).difference([n for n in screener_metrics.index]))
print(missing_screener_metrics)

for r in range(0,len(missing_screener_metrics)):
    r_ = missing_screener_metrics[r]
    test = pd.DataFrame([np.repeat('missing',len(screener_metrics.columns))],index=[r_],columns=screener_metrics.columns)
    screener_metrics = pd.concat([screener_metrics,test],axis=0)

#pd.DataFrame(screener_metrics)

['JW-A']


In [80]:
#fundamentals_quarterlies = pickle.load(open('./fundamental.pkl', 'rb'))
fundamental_entries = [e[0] for e in pickle_dumps]#[e[0] for e in fundamentals_quarterlies]


In [81]:
#summary = pd.concat([prices_df[prices_df['Symbol']==name_][['Symbol','MIN_Lookback_One_Year', 'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months','risk_trend_factor','risk_per_share','volume_factor']].iloc[[-1]].reset_index().set_index('Symbol') for name_ in np.unique(prices_df['Symbol'].values)]).reset_index()

In [82]:

#threshold = 0 
#stock_fundamentals = market_data
#screener = screener_metrics
#stock_indexes = sp1500_index_df


#spdr_indexes = indexes
#spdr_indexes.columns=['Symbol']
#vanguard_sectors = sectors
#vanguard_sectors.columns=['Symbol']
screener_metrics_w_sp1500index = screener_metrics.reset_index().rename(columns={'index': 'Symbol'}).merge(sp1500_index_df,on='Symbol',how='left')

screener_metrics_w_sp1500index_good = screener_metrics_w_sp1500index[screener_metrics_w_sp1500index['50td_tvf_vel']!='missing']

#risk_trend_threshold = screener_metrics_w_sp1500index_good['50td_tvf_vel'].quantile(q=[threshold]).values[0]
stock_sectors = market_data['sector']

temp = stock_sectors.reset_index()
temp.columns = ['Symbol','Sector']
sector_merged = screener_metrics.reset_index().rename(columns={'index': 'Symbol'}).merge(temp, on='Symbol', how='right')
temp_sm = sector_merged.merge(sp1500_index_df, on='Symbol', how='left').set_index('Symbol')
#temp_sm.columns = ['risk_trend_factor', 'Sector', 'index']

temp_index_symbol_map = pd.concat([sp1500_index_df,pd.DataFrame([invert_dict_indexes[s] for s in sp1500_index_df['Market Index']])],axis=1)
temp_index_symbol_map.columns = ['Symbol','Market Index','Index Symbol']

index_sector_merged = temp_sm.merge(temp_index_symbol_map.iloc[:,[0,2]], on='Symbol', how='left')

"""
for spdr_ in range(0,len(indexes)):
    spdr__ = indexes[spdr_]
    #spdr__i = indexes[spdr_]
    #temp_sm.iloc[np.where(temp_sm.index==spdr__)][['index']] = spdr__i
    temp_sm.iloc[np.where(temp_sm.index==spdr__)][['index_Symbol']] = spdr__
    """

"\nfor spdr_ in range(0,len(indexes)):\n    spdr__ = indexes[spdr_]\n    #spdr__i = indexes[spdr_]\n    #temp_sm.iloc[np.where(temp_sm.index==spdr__)][['index']] = spdr__i\n    temp_sm.iloc[np.where(temp_sm.index==spdr__)][['index_Symbol']] = spdr__\n    "

In [83]:


sector_performance = []
for v in sectors:    
    #sector_performance.append([v,screener_metrics.loc[[v]]['risk_trend_factor'].values[0]])
    sector_performance.append([v,np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[v]]['50td_tvf_vel']),np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[v]]['20td_tvf_vel']),np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[v]]['risk_trend_factor'])])
temp_v = pd.concat([pd.DataFrame([dict_sectors_reverse[v] for v in sectors]),pd.DataFrame(sectors)],axis=1)
temp_v.columns = ['Sector','Symbol']
sectors_risk_sorted = pd.DataFrame(sector_performance,columns=['Symbol','50td_tvf_vel','20td_tvf_vel','risk_trend_factor']).merge(temp_v, on='Symbol', how='left').sort_values(by='50td_tvf_vel',ascending=False)    
sectors_risk_sorted

,Symbol,50td_tvf_vel,20td_tvf_vel,risk_trend_factor,Sector
7,VGT,2.830643,1.415321,-4.289931,Technology
6,VIS,2.249404,1.124702,-3.266975,Industrials
0,VOX,1.826495,0.913247,-6.480635,Communication Services
1,VCR,1.711295,0.855648,-5.787061,Consumer Cyclical
4,VFH,1.655982,0.827991,-3.601769,Financial Services
8,VAW,1.532338,0.766169,-3.482537,Basic Materials
5,VHT,1.427777,0.713888,-5.494344,Healthcare
9,VNQ,0.880169,0.440085,-4.488883,Real Estate
10,VPU,0.871175,0.435587,-0.760400,Utilities
2,VDC,0.591445,0.295722,-2.840649,Consumer Defensive


In [84]:
index_performance = []
for v in indexes:
    #index_performance.append([v,screener_metrics.loc[[v]]['20td_tvf_vel'].values[0]])
    if(v =='SPTM'):
        index_performance.append([v,np.nanmean(index_sector_merged['50td_tvf_vel'].replace(['error','missing'], np.nan))])
    else:
        index_performance.append([v,np.nanmean(index_sector_merged[index_sector_merged['Market Index']==dict_indexes[v]]['50td_tvf_vel'].replace(['error','missing'], np.nan))])
    
    
temp_i = pd.DataFrame(index_performance,index=indexes)
temp_i.columns = ['Symbol','50td_tvf_vel']
index_risk_sorted = pd.DataFrame(index_performance,columns=['Symbol','50td_tvf_vel'],index=[dict_indexes[i] for i in indexes]).sort_values(by='50td_tvf_vel',ascending=False).reset_index()
index_risk_sorted.columns = ['Index','Symbol','50td_tvf_vel']
index_risk_sorted

,Index,Symbol,50td_tvf_vel
0,SP500,SPY,1.795594
1,SP400,MDYG,1.709231
2,SP1500,SPTM,1.698249
3,SP600,SLY,1.561270


In [85]:
risk_screened = screener_metrics_w_sp1500index_good[(screener_metrics_w_sp1500index_good['volume_factor']!= 'missing')]
risk_screened = risk_screened.merge(temp, on='Symbol', how='left')
print(risk_screened.columns)
#temp_df = risk_screened[['Symbol','Adj Close','MIN_Lookback_One_Year', 'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months','risk_trend_factor','volume_factor','adf','hurst','Sector','Market Index']]
#temp_df.set_index('Symbol',inplace=True)
#pd.concat([temp_df,temp_sm],axis=0)

#screened_final_set = risk_screened

Index(['Symbol', 'Date', 'buy_volume_pct', 'sell_volume_pct', 'daily_buy_vol',
       'daily_sell_vol', 'net_daily_vol', '20td_trend_vol',
       '20td_tot_daily_vol', '20td_trend_vol_factor', '20td_tvf_rng_div_2',
       '20td_tvf_vel', '50td_trend_vol', '50td_tot_daily_vol',
       '50td_trend_vol_factor', '50td_tvf_rng_div_4', '50td_tvf_vel',
       'MIN_Lookback_One_Year', 'MIN_Lookback_2_Months',
       'MIN_Lookback_1_Months', 'risk_trend_factor', 'risk_per_share',
       'volume_factor', 'Adj Close', '50tdBOLD', '50tdMA-TP', '50tdBOLU',
       '200tdSMA', '200tdSDev', 'adf_50td', 'tau_50td', 'tau_50td_p',
       'hurst_50td', 'Market Index', 'Sector'],
      dtype='object')


In [86]:
risk_screened
#risk_screened['20td_tvf_vel']

,Symbol,Date,buy_volume_pct,sell_volume_pct,daily_buy_vol,daily_sell_vol,net_daily_vol,20td_trend_vol,20td_tot_daily_vol,20td_trend_vol_factor,...,50tdMA-TP,50tdBOLU,200tdSMA,200tdSDev,adf_50td,tau_50td,tau_50td_p,hurst_50td,Market Index,Sector
0,LGIH,2022-08-12 00:00:00,0.455148,-0.544852,60170.506962,-72029.493038,-11858.986077,1027955.166794,4866300.0,0.21124,...,97.193801,118.18564,116.71875,22.533211,0.868538,0.613562,0.0,NaN,SP600,Consumer Cyclical
1,ADI,2022-08-12 00:00:00,0.889625,-0.110375,2845286.65984,-353013.34016,2492273.31968,19861883.859073,62369400.0,0.318456,...,158.6566,181.016687,163.719646,10.745469,0.907955,0.436735,0.000008,NaN,SP500,Technology
2,GDEN,2022-08-12 00:00:00,0.313252,-0.686748,56698.564659,-124301.435341,-67602.870681,-56257.816086,2886000.0,-0.019493,...,42.372133,48.514715,47.999,5.266001,0.150378,-0.066966,0.492749,NaN,SP600,Consumer Cyclical
3,YETI,2022-08-12 00:00:00,0.897196,-0.102804,641764.056985,-73535.943015,568228.11397,-2844300.400319,31228200.0,-0.091081,...,46.182533,51.403024,63.5881,18.745312,0.042927,0.182041,0.062131,NaN,SP400,Consumer Cyclical
4,KNX,2022-08-12 00:00:00,0.87931,-0.12069,896808.736344,-123091.263656,773717.472689,6772688.326678,32027500.0,0.211465,...,49.320467,56.008846,52.898501,4.858071,0.989341,0.656046,0.0,NaN,SP400,Industrials
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,AVNS,2022-08-12 00:00:00,0.836957,-0.163043,278455.577232,-54244.422768,224211.154464,134446.677535,5149000.0,0.026111,...,27.335747,29.494235,30.5218,2.793472,0.987866,0.457704,0.000003,NaN,SP600,Healthcare
1515,EOG,2022-08-12 00:00:00,0.90909,-0.09091,2112543.763905,-211256.236095,1901287.52781,14128408.291087,73447400.0,0.192361,...,113.537333,141.912942,108.907186,15.305982,0.346219,-0.409963,0.000027,NaN,SP500,Energy
1516,LNC,2022-08-12 00:00:00,0.910891,-0.089109,1273790.151303,-124609.848697,1149180.302607,10907.980016,35126200.0,0.000311,...,49.385533,55.312473,62.053316,8.942125,0.067785,-0.094733,0.331868,NaN,SP500,Financial Services
1517,UFPI,2022-08-12 00:00:00,0.620192,-0.379808,183576.891758,-112423.108242,71153.783516,1933637.926845,7420500.0,0.260581,...,76.9826,94.574185,81.209955,6.769919,0.923104,0.483463,0.000001,NaN,SP600,Basic Materials


In [87]:
#threshold_value = screener_metrics_w_sp1500index_good.dropna()['50td_tvf_vel'].quantile([threshold])

In [88]:
_50td_tvf_vel_sector_dict = dict()
_20td_tvf_vel_sector_dict = dict()
_risk_trend_factor_sector_dict = dict()
for s in sectors:
    dict_sectors_reverse[s]
    #_20td_tvf_vel_sector_dict[s] = screener_metrics.loc[[s]]['20td_tvf_vel'].values[-1]
    _50td_tvf_vel_sector_dict[s] = np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[s]]['50td_tvf_vel'])
    _20td_tvf_vel_sector_dict[s] = np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[s]]['20td_tvf_vel'])
    _risk_trend_factor_sector_dict[s] = np.mean(index_sector_merged[index_sector_merged['Sector']==dict_sectors_reverse[s]]['risk_trend_factor'])
    
_50td_tvf_vel_sector_dict['error']='error'
_20td_tvf_vel_sector_dict['error']='error'
_risk_trend_factor_sector_dict['error']='error'

In [89]:
"""
inv_dict_map = {v: k for k, v in dict_indexes.items()}
inv_dict_map[np.nan] = 'error'
in_ = [inv_dict_map[t[0]] for t in temp_sm[['Market Index']].values]
se_ = [dict_sectors[t[0]] for t in temp_sm[['Sector']].values]
#temp_sm[['Sector','index']]
"""
#symbols_w_index_sector = pd.concat([pd.DataFrame(se_,columns=['index'],index=temp_sm.index),pd.DataFrame(in_,columns=['sector'],index=temp_sm.index)],axis=1)


"\ninv_dict_map = {v: k for k, v in dict_indexes.items()}\ninv_dict_map[np.nan] = 'error'\nin_ = [inv_dict_map[t[0]] for t in temp_sm[['Market Index']].values]\nse_ = [dict_sectors[t[0]] for t in temp_sm[['Sector']].values]\n#temp_sm[['Sector','index']]\n"

In [90]:
#symbols_w_index_sector

In [91]:
#pd.DataFrame([dict_sectors[s] for s in index_sector_merged['Sector']],columns=['Sector Symbol'],index=index_sector_merged['Symbol']
sector_symbols = pd.DataFrame([dict_sectors[s] for s in index_sector_merged['Sector']],index=index_sector_merged['Symbol'],columns=['Sector Symbol'])

a = pd.DataFrame([_50td_tvf_vel_sector_dict[s] for s in sector_symbols['Sector Symbol']],columns=['sector_50td_tvf_vel'],index=sector_symbols.index)
b = pd.DataFrame([_20td_tvf_vel_sector_dict[s] for s in sector_symbols['Sector Symbol']],columns=['sector_20td_tvf_vel'],index=sector_symbols.index)
c = pd.DataFrame([_risk_trend_factor_sector_dict[s] for s in sector_symbols['Sector Symbol']],columns=['sector_risk_trend_factor'],index=sector_symbols.index)

#screener_final = pd.concat([screener_metrics,symbols_w_index_sector.set_index),symbols_w_sector_rtf,index_sector_merged[['Market Index','Sector']]],axis=1)

symbols_w_sector_rtf = pd.concat([a,b,c],axis=1)

In [92]:
screener_final = pd.concat([screener_metrics,symbols_w_sector_rtf,index_sector_merged.set_index('Symbol')[['Sector']],sector_symbols,index_sector_merged.set_index('Symbol')[['Market Index','Index Symbol']],market_data.loc[:, ~market_data.columns.isin(["sector"])]],axis=1)

a = screener_final[screener_final['50td_tvf_vel'].apply(type) != np.str_].sort_values(by='50td_tvf_vel',ascending=False)
b = screener_final[screener_final['50td_tvf_vel'].apply(type) == np.str_]
screener_sorted = pd.concat([a,b],axis=0)

In [93]:
#pd.concat([screener_metrics.sort_values(by='risk_trend_factor',ascending=False).iloc[:,1:],symbols_w_index_sector,symbols_w_sector_rtf,temp_sm[['Market Index','Sector']]],axis=1)

In [94]:
list_sector_n_indexes = []
list_stocks = []
for i in index_risk_sorted['Symbol'].values:
    list_sector_n_indexes.append(i)
for i in sectors_risk_sorted['Symbol'].values:
    list_sector_n_indexes.append(i)
#for i in screened_final_set['Symbol'].values:
for i in sp1500_index_df['Symbol'].values:
    list_stocks.append(i)
stocks_ = list_stocks
#stocks_ = np.unique(list_stocks, return_index=True)[1]
#list__ = [list_[symbol] for symbol in sorted(stocks_)]
#list__ = ['MCK','MPC','APA','LNTH','MUSA','CIVI','VAW','VGT','EQT','XOM','DVN','MRO','EOG','HES','CEIX','SM','RRC','MUR','MTDR','PXD','OXY','PDCE','NFG','FANG','TWI','PBF','MANT','HRB','VDC','ACC','GO','TVTY','FCN','VIVO']

In [95]:
#screener_sorted.head(50)

In [96]:
screener_sorted.to_excel(writer, sheet_name='screener_sorted')


screener_sorted.to_csv('..\\data\\processed\\'+end.strftime('%Y-%m-%d')+'_screener_sorted.csv')
screener_sorted.to_csv('..\\data\\processed\\screener_sorted.csv')



In [97]:
#screener_final.loc[np.setdiff1d([*screener_final.index],list_sector_n_indexes)]['risk_trend_factor']

In [98]:
writer.close()

C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [99]:
npa = []
for f in FRED_Indicators:
    npa.append([f,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d'),nyse_dates])

In [100]:
completed_fred = pd.DataFrame()

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(Fred_Data, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_fred = pd.concat([completed_fred,f.result()],axis=0)
client.close()

completed_fred_pvt = pd.pivot_table(completed_fred, values=['Value'], index=completed_fred.index,columns=['Symbol'])
completed_fred_pvt

C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\distributed\client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.10.6 | None      | None    |
| lz4     | 4.0.2  | 3.1.10    | 3.1.10  |
| numpy   | 1.23.1 | 1.22.4    | 1.22.4  |
| tornado | 6.2    | 6.1       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Value                                                   \
Symbol     ASPUS AWHAETP B432RG3A086NBEA BAA10Y BACDINA066MNFRBNY   
2017-04-17   NaN     NaN             NaN   2.29               NaN   
2017-04-18   NaN     NaN             NaN   2.29               NaN   
2017-04-19   NaN     NaN             NaN   2.29               NaN   
2017-04-20   NaN     NaN             NaN   2.28               NaN   
2017-04-21   NaN     NaN             NaN   2.29               NaN   
...          ...     ...             ...    ...               ...   
2022-08-09   NaN     NaN             NaN   2.26               NaN   
2022-08-10   NaN     NaN             NaN   2.29               NaN   
2022-08-11   NaN     NaN             NaN   2.29               NaN   
2022-08-12   NaN     NaN             NaN   2.27               NaN   
2022-08-15   NaN     NaN             NaN    NaN               NaN   

                                                                       \
Symbol     BACTSAMFRBDAL BAMLC0A0CM BAMLCC0A1AAATRIV BAMLCC0A4BBBTRIV   
2017-04-17           NaN       1.24           605.16           783.96   
2017-04-18           NaN       1.25           609.38           787.34   
2017-04-19           NaN       1.25           608.22           786.34   
2017-04-20           NaN       1.25           606.65           784.75   
2017-04-21           NaN       1.25           606.70           785.16   
...                  ...        ...              ...              ...   
2022-08-09           NaN       1.50           677.84           879.47   
2022-08-10           NaN       1.47           678.35           881.96   
2022-08-11           NaN       1.42           673.06           878.88   
2022-08-12           NaN       1.41           675.70           882.40   
2022-08-15           NaN        NaN              NaN              NaN   

                        ...                                                 \
Symbol     BAMLH0A3HYC  ... VIXCLS VXVCLS WDFUELLA    WGS3MO WILLLRGCAPVAL   
2017-04-17        8.58  ...  14.66  14.86      NaN       NaN      71071.70   
2017-04-18        8.71  ...  14.42  14.76      NaN       NaN      70878.81   
2017-04-19        8.67  ...  14.93  15.08      NaN       NaN      70624.33   
2017-04-20        8.68  ...  14.15  14.53      NaN       NaN      71011.40   
2017-04-21        8.68  ...  14.63  14.75    1.642  0.810000      70782.69   
...                ...  ...    ...    ...      ...       ...           ...   
2022-08-09       10.15  ...  21.77  24.69      NaN  2.597143     119486.96   
2022-08-10        9.94  ...  19.74  23.46      NaN  2.611429     121456.09   
2022-08-11        9.75  ...  20.20  23.71      NaN  2.625714     122083.26   
2022-08-12        9.77  ...  19.53  23.31      NaN  2.640000     123832.13   
2022-08-15         NaN  ...    NaN    NaN      NaN       NaN           NaN   

                                                                
Symbol     WILLMICROCAPPR WPU0911 WPU101 WPUSI019011    WTB3MS  
2017-04-17       12370.71     NaN    NaN         NaN       NaN  
2017-04-18       12351.63     NaN    NaN         NaN       NaN  
2017-04-19       12385.23     NaN    NaN         NaN       NaN  
2017-04-20       12518.76     NaN    NaN         NaN       NaN  
2017-04-21       12502.37     NaN    NaN         NaN  0.800000  
...                   ...     ...    ...         ...       ...  
2022-08-09       14186.05     NaN    NaN         NaN  2.511429  
2022-08-10       14544.88     NaN    NaN         NaN  2.524286  
2022-08-11       14564.31     NaN    NaN         NaN  2.537143  
2022-08-12       14888.73     NaN    NaN         NaN  2.550000  
2022-08-15            NaN     NaN    NaN         NaN       NaN  

[1343 rows x 145 columns]

In [101]:
npa = []
for e in etf_bonds:
    npa.append([e,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d'),nyse_dates])
    
completed_bonds = pd.DataFrame()

client = Client('192.168.3.100:8786')
#client = Client(n_workers=4,threads_per_worker=1)
client.restart()

future = client.map(dl, npa)

#my intent was to capture future objects vs results and this gave me resultsa
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_bonds = pd.concat([completed_bonds,f.result()],axis=0)
client.close()

completed_bonds_pvt = pd.pivot_table(completed_bonds, values='Close', index=completed_bonds.index,columns=['Symbol'])
completed_bonds_pvt

C:\Users\User\AppData\Local\Programs\3.9-JupyterLab\lib\site-packages\distributed\client.py:1131: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.10.6 | None      | None    |
| lz4     | 4.0.2  | 3.1.10    | 3.1.10  |
| numpy   | 1.23.1 | 1.22.4    | 1.22.4  |
| tornado | 6.2    | 6.1       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Symbol,AGG,AGZ,BIL,GBF,GOVT,IEF,IEI,ILTB,IMTB,ISTB,IUSB,LQD,MEAR,MUB,NEAR,SHV,SHY,SUB,TLT
2017-04-17,98.323402,104.521568,87.308167,103.669724,23.343380,99.322266,115.862526,53.378517,44.131664,45.503426,45.289539,104.106239,47.738674,99.490952,46.306110,104.705963,79.939926,100.244499,112.322708
2017-04-18,98.719543,104.696411,87.327286,104.079712,23.454363,99.889694,116.255493,53.703335,44.158337,45.521553,45.378624,104.719131,47.733917,99.790443,46.333775,104.705963,80.006027,100.282425,113.791878
2017-04-19,98.566505,104.724014,87.327286,103.970413,23.408123,99.629242,116.049629,53.660591,44.060452,45.521553,45.369705,104.403954,47.610016,99.717857,46.333775,104.724922,79.968239,100.320358,113.171402
2017-04-20,98.413429,104.530708,87.327286,103.706184,23.371124,99.405983,115.928024,53.446896,44.007053,45.494392,45.342976,104.255081,47.733917,99.599854,46.315334,104.705963,79.958763,100.263458,112.733345
2017-04-21,98.440453,104.622787,87.327286,103.669724,23.371124,99.452507,116.012215,53.532379,44.140541,45.530598,45.334072,104.255081,47.733917,99.618027,46.324558,104.715431,79.996544,100.339310,112.733345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-08,103.180000,110.290001,91.430000,108.959999,24.090000,104.190002,119.989998,57.660000,45.650002,47.560001,47.630001,113.470001,49.810001,107.910004,49.279999,109.980003,82.550003,105.220001,118.349998
2022-08-09,102.949997,110.290001,91.419998,108.790001,24.040001,103.879997,119.709999,57.369999,45.535000,47.450001,47.480000,112.959999,49.840000,107.900002,49.259998,109.980003,82.470001,105.180000,117.889999
2022-08-10,103.209999,110.180000,91.430000,108.930000,24.040001,103.940002,119.889999,57.599998,45.750000,47.570000,47.650002,114.029999,49.810001,108.120003,49.279999,110.010002,82.550003,105.150002,117.120003
2022-08-11,102.660004,110.099998,91.430000,108.279999,23.910000,103.250000,119.529999,56.639999,45.549999,47.540001,47.419998,112.930000,49.820000,107.790001,49.259998,110.029999,82.519997,105.099998,114.389999


In [102]:
#pd.pivot_table(completed_bonds, values='Close', index=completed_bonds.index,columns=['Symbol'])

In [103]:
def return_vwap(data):
    data['VWAP'] = data['Volume'] * data['Adj Close']
    return(data)

In [104]:
prices_df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol']]

,Open,High,Low,Close,Adj Close,Volume,Symbol
2017-04-17,17.010000,17.379999,16.950001,17.340000,14.458384,8082500.0,KEY
2017-04-18,17.170000,17.370001,17.020000,17.200001,14.341647,10552700.0,KEY
2017-04-19,17.320000,17.580000,17.230000,17.370001,14.483394,11069700.0,KEY
2017-04-20,17.870001,18.469999,17.809999,18.209999,15.183800,16549100.0,KEY
2017-04-21,18.090000,18.350000,17.910000,18.150000,15.133770,13161300.0,KEY
...,...,...,...,...,...,...,...
2022-08-08,22.940001,23.690001,22.500000,23.459999,23.459999,86000.0,ANIK
2022-08-09,23.290001,23.340000,22.379999,22.559999,22.559999,67100.0,ANIK
2022-08-10,22.969999,23.490000,22.780001,23.000000,23.000000,99300.0,ANIK
2022-08-11,23.070000,23.070000,21.629999,21.740000,21.740000,122800.0,ANIK


In [105]:
return_vwap(prices_df)

,Open,High,Low,Close,Adj Close,Volume,Symbol,VWAP,buy_volume_pct,sell_volume_pct,...,50tdBOLU,50tdBOLD,trailing_risk_40d_max,trailing_risk_40d_min,risk_per_share,30d_vol,30d_vol_2yr,volume_factor,1YMF,risk_trend_factor
2017-04-17,17.010000,17.379999,16.950001,17.340000,14.458384,8082500.0,KEY,1.168599e+08,-5.794480,-6.794480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-18,17.170000,17.370001,17.020000,17.200001,14.341647,10552700.0,KEY,1.513431e+08,-7.652430,-8.652430,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-19,17.320000,17.580000,17.230000,17.370001,14.483394,11069700.0,KEY,1.603268e+08,-7.847437,-8.847437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-20,17.870001,18.469999,17.809999,18.209999,15.183800,16549100.0,KEY,2.512782e+08,-3.979091,-4.979091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-21,18.090000,18.350000,17.910000,18.150000,15.133770,13161300.0,KEY,1.991801e+08,-6.309606,-7.309606,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-08,22.940001,23.690001,22.500000,23.459999,23.459999,86000.0,ANIK,2.017560e+06,0.806722,-0.193278,...,24.063800,20.935267,25.209999,20.120001,5.089998,1448200.0,2.137721e+06,0.677450,-67.050007,-13.172894
2022-08-09,23.290001,23.340000,22.379999,22.559999,22.559999,67100.0,ANIK,1.513776e+06,0.187500,-0.812500,...,24.070105,20.970695,25.209999,20.120001,5.089998,1450900.0,2.135939e+06,0.679280,-69.750006,-13.703346
2022-08-10,22.969999,23.490000,22.780001,23.000000,23.000000,99300.0,ANIK,2.283900e+06,0.309859,-0.690141,...,24.091139,20.998595,25.209999,20.170000,5.039999,1500000.0,2.134254e+06,0.702822,-68.430004,-13.577384
2022-08-11,23.070000,23.070000,21.629999,21.740000,21.740000,122800.0,ANIK,2.669672e+06,0.076389,-0.923611,...,24.085537,21.022063,25.209999,20.510000,4.699999,1583100.0,2.132935e+06,0.742217,-72.210005,-15.363835


In [106]:
writer = pd.ExcelWriter('SP1500_screener_multiple.xlsx', engine='xlsxwriter')

data_object = [start,end,prices_df, sp1500_index_df, sp500, sp600, sp400, market_data, completed_fred_pvt, completed_bonds, completed_bonds_pvt, pd.DataFrame(sectors,sector_names), pd.DataFrame(indexes,index_names), screener_sorted, dict_sectors, dict_indexes, dict_sectors_reverse,invert_dict_indexes, list_sector_n_indexes, list_stocks, future_commodities_n_w_index_pvt_w_Fred,price_comm_df_pvt]
               
pickle.dump(data_object, open('..\\data\\interim\\data_object.pkl', 'wb'))
"""
#prices_df.to_excel(writer, sheet_name='SP1500_Sectors_Indexes_Prices')
#prices_df.drop(['Sector', 'Market Cap'], axis=1).to_excel(writer, sheet_name='SP1500Prices')

#sectors_df.to_excel(writer, sheet_name='stock_sectors')

#mktcap_df.to_excel(writer, sheet_name='market_caps')

sp1500_index_df.to_excel(writer, sheet_name='SP1500')

pd.DataFrame(sp500).to_excel(writer, sheet_name='SP500')
pd.DataFrame(sp600).to_excel(writer, sheet_name='SP600')
pd.DataFrame(sp400).to_excel(writer, sheet_name='SP400')
pd.DataFrame(market_data).to_excel(writer, sheet_name='Stock_Fundamentals')
completed_fred_pvt.to_excel(writer, sheet_name='completed_fred_pvt')
completed_bonds.to_excel(writer, sheet_name='completed_bonds')
completed_bonds_pvt.to_excel(writer, sheet_name='completed_bonds_pvt')
pd.DataFrame(pd.DataFrame(sectors,sector_names)).to_excel(writer, sheet_name='vanguard_sectors')
pd.DataFrame(pd.DataFrame(indexes,index_names)).to_excel(writer, sheet_name='spdr_indexes')
screener_metrics.to_excel(writer, sheet_name='Screener')

writer.save()

wb = openpyxl.load_workbook('SP1500_screener_multiple.xlsx')
#openpyxl.Workbook()
new_range = openpyxl.workbook.defined_name.DefinedName('Prices', attr_text='SP1500_Sectors_Indexes_Prices!$A$1:$H$'+str(len(prices_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SP1500', attr_text='SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Vanguard_Sectors', attr_text='vanguard_sectors!$B$1:$C$'+str(len(sectors)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SPDR_Indexes', attr_text='spdr_indexes!$B$1:$C$'+str(len(indexes)+1))
wb.defined_names.append(new_range)


#wb.defined_names.append(new_range)
wb.save('SP1500_screener_multiple.xlsx')
wb.close()
#named_range = NamedRange("test_nr", [(new_sheet, "A1")])
#wb.add_named_range(new_range)
wb.close()
writer.close()
"""
"""
workbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')
workbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))
workbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
workbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
workbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))

"""
#workbook.close()

"\nworkbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')\nworkbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))\nworkbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))\nworkbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))\nworkbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))\n\n"